# <center>Deepseek企业级Agent项目开发实战</center>

In [138]:
print("Hello, World!")

Hello, World!


## <center>Part 3. Microsoft GraphRAG 自定义接入图数据库 Neo4j</center>

&emsp;&emsp;`Microsoft GraphRAG` 的核心概念在于将`Konwledge Graph` 存储的数据与大语言模型结合，通过查询图数据库来增强模型的回答质量。经过前两节课程对`Microsoft GraphRAG`构建索引过程的详解，我们已经借助大模型结合提示工程从非结构化文本中提取出实体和关系，其实这就足够支撑我们构建基于本地文档的完整知识图谱了。

&emsp;&emsp;因此，本节课，我们就详细的介绍一下如果将`Microsoft GraphRAG`构建的索引文件存储至本地的图数据库中进行可视化的展示，以便用于接下来的`GraphRAG Query`阶段。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503041644948.png" width=80%></div>

&emsp;&emsp;首先，我们需要在本地搭建一个图数据库来存储知识结构。

&emsp;&emsp; Neo4j 是一个开源的 `NoSQL` 图数据库，它使用图来表示和存储数据。在 `DB-Engines` 排名中根据数据库管理系统的受欢迎程度对其进行排名中，`Graph DBMS` 榜单中 `Neo4j` 排名第一。查询链接为：[Graph DBMS 排名](https://db-engines.com/en/ranking/graph+dbms?source=post_page-----1cb68f1ac2---------------------------------------)

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503121518292.png" width=60%></div>

&emsp;&emsp;在总体排名中，`Neo4j` 位列第十九，在技术市场上占有很大的空间。其开源地址为：[Neo4j 开源地址](https://github.com/neo4j/neo4j)

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503121522873.png" width=60%></div>

&emsp;&emsp;`Neo4j` 的安装方法非常多，比如`Docker`安装， `Neo4j Desktop` 安装，`源码`安装以及在线适用等方式。最简单的方式肯定是使用`Neo4j Desktop` 安装，只需要下载安装包，解压后便可以正常运行。下载安装包的地址如上图所示，在`Neo4j`的`GitHub`地址中找到`Download`按钮，下载`.exe`安装包（比如`Windows`系统），执行傻瓜式安装。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503121530623.png" width=60%></div>

&emsp;&emsp;这种安装方式及其简单，因为不需要我们配置任何环境变量，只需要解压后便可运行。但是其成功率并不是特别高，因为电脑环境的差异，可能会导致安装后出现各种未知的问题，比如打开后报错、闪退、点击没有任何反应等情况。因此，我们这里给大家讲解一种最稳定的方式：源码安装。

&emsp;&emsp;接下来，我们就基于`Win`系统来给大家详细讲解源码安装`Neo4j`的完整流程。

# 1. Windows 源码安装 Neo4j

&emsp;&emsp;首先，还是可以通过`GitHub`地址中的`Download`按钮进入到`Neo4j`的官方地址，也可以点击该链接直接进入：https://neo4j.com/

&emsp;&emsp;在最上方导航栏找到`Products`，点击`Deployment Center`，进入到`Neo4j`的源码下载页面。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503121518268.png" width=60%></div>

&emsp;&emsp;在`Neo4j`的源码下载页面中，需要选择`COMMUNITY`版本，(社区版，即免费版)，然后选择`Windows`版本，并且指定以什么方式下载，比如`tar`包，`unzip`包，`Source Code`，选择结束后，点击`Download`按钮，下载源码包。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503121518269.png" width=60%></div>

&emsp;&emsp;如果网络正常，会自动弹出窗口，可以选择下载至本地的路径。如下所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503121518270.png" width=60%></div>

&emsp;&emsp;最后，下载完成后，将`tar` 或者 `zip` 包解压后，便可以得到`Neo4j`的源码包。如下所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503121518271.png" width=60%></div>

&emsp;&emsp;至此，`Neo4j`的源码包便下载完成了，其实也并不复杂。但是在运行和启动`Neo4j`服务之前，我们需要先配置一下`Neo4j`的环境变量。配置方法就是常规的`Windows`环境变量配置，首先右键点击`此电脑`，选择`属性`，然后点击`高级系统设置`：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503121518272.png" width=60%></div>

&emsp;&emsp;在`系统属性`窗口中，点击`环境变量`按钮，进入到`环境变量`窗口。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503121518273.png" width=60%></div>

&emsp;&emsp;在系统变量中，点击`新建`按钮：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503121518274.png" width=60%></div>

&emsp;&emsp;在`新建`窗口中，输入变量名`NEO4J_HOME`，变量值为`Neo4j`的安装路径，比如我刚才解压的文件路径就是`D:\neo4k-community-2024.02.0-windows`，然后点击`确定`按钮：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503121518275.png" width=60%></div>

&emsp;&emsp;配置好`NEO4J_HOME`环境变量后，我们还需要配置`Path`环境变量。在`系统变量`中，点击`Path`变量，然后点击`编辑`按钮:

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503121518276.png" width=60%></div>

&emsp;&emsp;在`Path`变量中，点击`新建`按钮，然后输入`%NEO4J_HOME%\bin`，然后点击`确定`按钮：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503121518278.png" width=60%></div>

&emsp;&emsp;配置好`Path`环境变量后，`Neo4j`服务就配置好了。但在启动之前，我们还需要检查一下当前系统是否存在`Java`环境，以及`Java`版本是否满足要求。首先需要明确的是，`Neo4j`服务依赖于`Java`环境，同时对`Java`版本也有一定的要求，要求`Java`版本是`OpenJDK`的`21`版本。官方说明：https://neo4j.com/docs/operations-manual/current/installation/windows/

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503121558112.png" width=60%></div>

&emsp;&emsp;检查当前系统`Java`环境是否存在，以及`Java`版本是否满足要求的命令如下所示：在`cmd`命令行中输入`java -version`，然后回车，如果出现`Java`版本信息，则说明`Java`环境存在，并且版本满足要求。如果出现`Java`环境不存在，或者版本不满足要求，则需要先安装`Java`环境。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503121518286.png" width=60%></div>

&emsp;&emsp;如果`Java`环境存在，并且版本满足要求，则可以跳过接下来`Java`环境的安装，直接启动`Neo4j`服务。如果`Java`环境不存在，或者版本不满足要求，则需要先安装`Java`环境。具体的方法为：首先进入`Oracle`官网，点击`Java`，然后点击`Java SE`，进入到`Java`的下载页面。 地址：https://www.oracle.com/java/technologies/downloads/?er=221886#java21

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503121606257.png" width=60%></div>

&emsp;&emsp;下载至本地并执行`.exe` 的安装即可，这里不在赘述。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503121518282.png" width=60%></div>

&emsp;&emsp;同样，安装完成以后，也需要配置`JAVA_HOME`环境变量。在`系统变量`中，点击`新建`按钮，然后输入`JAVA_HOME`，变量值为`Java`的安装路径，比如我刚才安装的`Java`路径是`C:\Program Files\Java\jdk-21`，然后点击`确定`按钮：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503121611802.png" width=80%></div>

&emsp;&emsp;配置好`JAVA_HOME`环境变量后，我们还需要配置`Path`环境变量。在`系统变量`中，点击`Path`变量，然后点击`编辑`按钮：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503121518285.png" width=60%></div>

&emsp;&emsp;全部配置好以后，在`cmd`命令行中输入`java -version`，然后回车，如果出现`Java`版本信息，则说明`java`环境配置成功。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503121518286.png" width=60%></div>

&emsp;&emsp;至此，配置好了`Neo4j`服务的环境，同时其依赖的`Java`环境也配置好了。接下来，我们就可以启动`Neo4j`服务了。在`cmd`命令行中输入`neo4j windows-service install`，然后回车，如果出现`Neo4j`服务安装成功的信息，则说明`Neo4j`服务安装成功。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503121518287.png" width=60%></div>

&emsp;&emsp;安装成功后，在`cmd`命令行中输入`neo4j.bat console`，然后回车，如果出现`Neo4j`服务启动成功的信息，则说明`Neo4j`服务启动成功。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503121518288.png" width=80%></div>

&emsp;&emsp;启动后会看到两个连接地址，一个是`Bolt:localhost:7687`，一个是`HTTP:localhost:7474`，其中`Bolt`是`Neo4j`的`Bolt`协议连接地址，`HTTP`是`Neo4j`的`HTTP`协议连接地址。我们可以通过`localhost:7474`访问`Neo4j`的`Web`界面，如下所示：

&emsp;&emsp;首次登录的默认账号是`neo4j`，密码是`neo4j`。输入后点击`connect`按钮，即可进入到`Neo4j`的`Web`界面。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503121518289.png" width=80%></div>

&emsp;&emsp;同时，首次登录成功后会提示修改密码，此时我们点击`Change Password`按钮，然后输入新的密码，点击`Save`按钮，即可修改密码。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503121518290.png" width=80%></div>

&emsp;&emsp;至此，`Neo4j`服务便在本地安装成功并顺利了，接下来就可以开始使用`Neo4j`服务了。

# 2. Python 连接 Neo4j

&emsp;&emsp;`Neo4j` 与 `Python`、`JavaScript`、`Java` 和 `.Net` 等库都做了集成，因此只要安装了对应的库，就可以连接`Neo4j`服务。在`Python`中，可以直接通过`pip` 安装`neo4j`包来连接`Neo4j`服务。执行如下命令：

In [108]:
! pip install neo4j

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


&emsp;&emsp;从`neo4j`包中导入`GraphDatabase`类，然后设置`Neo4j`的连接地址、用户名、密码和数据库名称，然后创建`driver`对象，最后通过`driver`对象连接`Neo4j`服务。执行如下代码：

In [139]:
from neo4j import GraphDatabase


NEO4J_URI="bolt://localhost"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="thy382324"  # 这里替换成自己设置的密码
NEO4J_DATABASE="neo4j"         # 社区版仅能使用默认的neo4j数据库，不支持创建其他数据库

driver = GraphDatabase.driver(
    NEO4J_URI, 
    auth=(NEO4J_USERNAME, NEO4J_PASSWORD)
    )

&emsp;&emsp;可以快速做一个连接测试，执行如下代码：

In [140]:
driver = GraphDatabase.driver(
    NEO4J_URI, 
    auth=(NEO4J_USERNAME, NEO4J_PASSWORD)
    )

def test_connection():
    # 在这里创建会话
    with driver.session() as session:
        session.run("MATCH (n) RETURN n LIMIT 1")

try:
    test_connection()
    print("连接成功！")
except Exception as e:
    print("连接失败:", e)
finally:
    driver.close()  # 确保在所有操作完成后再关闭驱动程序

连接成功！


&emsp;&emsp;如果返回结果为`连接成功！`，则说明`Neo4j`服务一切正常，便可以进入下面的实践。否则需要进一步检查`Neo4j`服务是否正常启动，以及`Neo4j`的连接地址、用户名、密码和数据库名称是否正确。

# 3. 知识图谱快速入门

&emsp;&emsp;知识图谱建立在图论的基础上，而图是用于对对象之间的成对关系进行建模的结构。图主要由两个主要元素组成：<font color="red">节点</font>和<font color="red">关系</font>。其中：

- 节点代表单个对象或实体，类似于传统数据库中的记录。这些实体可以是任何人、公司或地点。
- 节点通过标签进行分类，方便根据节点的角色进行分类和查询，例如“客户”或“产品”。

&emsp;&emsp;`Neo4j` 是存储知识图谱的其中一种图数据库，它支持节点和关系的存储，同时，通过`Cypher`查询语言进行图谱的创建、查询和分析，即 `Cypher` 是 `Neo4j` 中用于交互和操作图谱的查询语言。其形式及如下：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503131043957.png" width=80%></div>

&emsp;&emsp;而关于`Cypher`查询语言的详细介绍，可以参考`Neo4j`的官方文档：[Cypher Core concepts](https://neo4j.com/docs/cypher-manual/current/queries/concepts/)进行学习。我们这里不对基础内容展开详细的介绍。本节课的重点是借助`Neo4j` 的 `Python` 集成，给大家介绍用于接下来`Microsoft GraphRAG` 索引文件接入过程时必备的相关概念。主要有以下几点：

- **节点，可以是`Person`、`Organization`、`Product`、`Location`等实体类型，每个节点可以有多个属性，例如`name`、`age`、`gender`等，同时<font color="red">属性是以键值对形式存储的</font>。**

In [141]:
from neo4j import GraphDatabase


NEO4J_URI="bolt://localhost"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="thy382324"  # 这里替换成自己设置的密码
NEO4J_DATABASE="neo4j"         # 社区版仅能使用默认的neo4j数据库，不支持创建其他数据库

driver = GraphDatabase.driver(
    NEO4J_URI, 
    auth=(NEO4J_USERNAME, NEO4J_PASSWORD)
    )

&emsp;&emsp;通过`CREATE`语句可以创建节点，同时可以为节点添加属性，语法形式是：

```python 
    CREATE (node_name:node_type {property_name: property_value, ...})
```

&emsp;&emsp;其中，`node_name` 这是一个变量名，用于引用创建的节点，`node_type` 是节点的标签，表示节点的类型，`property_name` 是属性的名称，`property_value` 是属性的值。

In [142]:

with driver.session() as session:
    # 添加多个节点：Person
    session.run("""
        CREATE (p1:Person {name: 'muyu', age: 30, city: 'beijing'}),
               (p2:Person {name: 'zhangsan', age: 23, city: 'shanghai'})
    """)
    
    # 添加多个节点：Company
    session.run("""
        CREATE (c1:Company {name: 'Fufan', industry: 'Technology', location: 'beijing'}),
               (c2:Company {name: 'Beyond', industry: 'Education', location: 'beijing'})
    """)


&emsp;&emsp;如上代码所示：类似 `p1` 这种的生命周期仅仅是局部作用域：变量的作用域通常是局部的，仅在创建它的查询中有效。一旦查询执行完毕，变量就不再存在。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503131057101.png" width=80%></div>

&emsp;&emsp;这里面的`Person` 和 `Company` 是节点的标签，`name`、`age`、`city` 是节点的属性，`muyu`、`zhangsan`、`beijing` 是节点的属性值。 其中：

- 通过`CREATE`语句创建节点时，如果该标签不存在，则创建该标签，否则不创建。
- 一个标签下可以有多个节点，一个节点也可以有多个标签。比如`Person` 标签下可以有`muyu` 和 `zhangsan` 两个节点， 同时`muyu` 和 `zhangsan` 也可以有多个标签，比如同时是 `Person` 和 `Student`。
- 一个节点可以有多个属性，比如`muyu` 节点可以有`name`、`age`、`city` 三个属性。

&emsp;&emsp;比如我们现在将`muyu` 和 `zhangsan` 增加到 `Student` 标签下的节点，同时他们也是 `Person` 标签下的节点。因此我们通过如下`Cypher`语句进行创建：

In [143]:
with driver.session() as session:
    # 为名为"muyu"的节点添加Student标签
    session.run("""
        MATCH (p:Person {name: 'muyu'})
        SET p:Student
        RETURN p
    """)
    
    # 为名为"zhangsan"的节点添加Student标签
    session.run("""
        MATCH (p:Person {name: 'zhangsan'})
        SET p:Student
        RETURN p
    """)

&emsp;&emsp;这里需要了解的知识点是：`MATCH` 语句用于在图谱中查找节点，`SET` 语句用于为节点添加标签，`RETURN` 语句用于返回添加标签后的节点。执行后在`Neo4j`的浏览器中可以看到：`muyu` 和 `zhangsan` 同时是 `Person` 和 `Student` 标签下的节点。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503131118642.png" width=80%></div>

&emsp;&emsp;同时，我们也可以更新节点的属性，比如将`muyu` 的`age` 属性更新为`31`，通过如下`Cypher`语句进行更新：

In [144]:
with driver.session() as session:
    session.run("""
        MATCH (p:Person {name: 'muyu'})
        SET p.age = 32
        RETURN p
    """)

&emsp;&emsp;执行后在`Neo4j`的浏览器中可以看到：`muyu` 的`age` 属性被更新为`31`。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503131121413.png" width=80%></div>

- 关系：关系是连接节点的边，用于表示节点之间的关联。

&emsp;&emsp;关系由两个主要元素组成：
  - 关系类型：表示关系的名称，例如“朋友”或“同事”。
  - 关系方向：表示关系的方向，例如“从A到B”或“从B到A”。

&emsp;&emsp;关系类型和关系方向是关系的重要组成部分，用于描述节点之间的关联。关系类型可以是任何字符串，而关系方向可以是“正向”或“反向”。其语法形式如下：

```python
    CREATE (node_name)-[:relation_type]->(related_node_name)
```

&emsp;&emsp;其中，`node_name` 是节点的名称，`relation_type` 是关系的类型，`related_node_name` 是相关节点的名称

&emsp;&emsp;这里我们以`muyu` 和 `zhangsan` 为例，创建`company` 和 `person` 之间的关系，通过如下`Cypher`语句进行创建：

In [145]:
with driver.session() as session:
    # 创建muyu与Fufan公司之间的EMPLOYED_BY关系
    session.run("""
        MATCH (p:Person {name: 'muyu'})
        MATCH (c:Company {name: 'Fufan'})
        CREATE (p)-[:EMPLOYED_BY]->(c)
    """)
    
    # 创建zhangsan与Beyond公司之间的LIVES_IN关系
    session.run("""
        MATCH (p:Person {name: 'zhangsan'})
        MATCH (c:Company {name: 'Beyond'})
        CREATE (p)-[:EMPLOYED_BY]->(c)
    """)

&emsp;&emsp;需要使用`MATCH`语句先找到相应的节点，然后再创建它们之间的关系。执行代码后，则可以在`Neo4j`的浏览器中看到：`muyu` 和 `zhangsan` 与`Fufan` 和 `Beyond` 公司之间建立了`EMPLOYED_BY` 关系。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503131127430.png" width=80%></div>

&emsp;&emsp;以上是通过节点、属性、标签、关系四个概念的角度，给大家介绍了`Neo4j` 中图谱的创建、更新和查询。这样的图谱其实就已经具备了知识图谱的雏形，即：

- 节点：表示实体，例如人、公司、地点等。
- 属性：表示实体的特征，例如姓名、年龄、城市等。
- 标签：表示实体的类型，例如`Person`、`Company`、`Student`等。
- 关系：表示实体之间的关系，例如`EMPLOYED_BY`等。

&emsp;&emsp;我们是可以基于这样的图谱开始进行简单到复杂的查询的，比如如下的一些示例：

In [146]:
def neo4j_query_examples(driver, query_type, params=None):
    """
    执行各种Neo4j查询示例
    
    参数:
    - driver: Neo4j驱动实例
    - query_type: 查询类型，可选值包括:
        'all_persons', 'all_companies', 'filter_by_city', 'all_relationships',
        'specific_relationship', 'node_relationships', 'path_query', 'aggregation',
        'group_by', 'colleagues', 'complex_query', 'param_query', 'subgraph', 'community'
    - params: 查询参数字典，根据查询类型不同而不同
    
    返回:
    - 查询结果列表
    """
    if params is None:
        params = {}
    
    results = []
    
    with driver.session() as session:
        if query_type == 'all_persons':
            # 查询所有Person节点
            result = session.run("""
                MATCH (p:Person)
                RETURN p.name AS name, p.age AS age, p.city AS city
            """)
            
            print("所有Person节点:")
            for record in result:
                print(f"姓名: {record['name']}, 年龄: {record['age']}, 城市: {record['city']}")
                results.append({
                    'name': record['name'],
                    'age': record['age'],
                    'city': record['city']
                })
        
        elif query_type == 'all_companies':
            # 查询所有Company节点
            result = session.run("""
                MATCH (c:Company)
                RETURN c.name AS name, c.industry AS industry, c.location AS location
            """)
            
            print("所有Company节点:")
            for record in result:
                print(f"公司: {record['name']}, 行业: {record['industry']}, 位置: {record['location']}")
                results.append({
                    'name': record['name'],
                    'industry': record['industry'],
                    'location': record['location']
                })
        
        elif query_type == 'filter_by_city':
            # 按城市过滤Person节点
            city = params.get('city', 'beijing')
            result = session.run("""
                MATCH (p:Person)
                WHERE p.city = $city
                RETURN p.name AS name, p.age AS age
            """, {'city': city})
            
            print(f"{city}的人员:")
            for record in result:
                print(f"姓名: {record['name']}, 年龄: {record['age']}")
                results.append({
                    'name': record['name'],
                    'age': record['age']
                })
        
        elif query_type == 'all_relationships':
            # 查询所有关系
            result = session.run("""
                MATCH (p:Person)-[r]->(c:Company)
                RETURN p.name AS person, type(r) AS relationship, c.name AS company
            """)
            
            print("所有人员与公司的关系:")
            for record in result:
                print(f"{record['person']} {record['relationship']} {record['company']}")
                results.append({
                    'person': record['person'],
                    'relationship': record['relationship'],
                    'company': record['company']
                })
        
        elif query_type == 'specific_relationship':
            # 查询特定类型的关系
            rel_type = params.get('rel_type', 'EMPLOYED_BY')
            result = session.run(f"""
                MATCH (p:Person)-[:{rel_type}]->(c:Company)
                RETURN p.name AS person, c.name AS company
            """)
            
            print(f"{rel_type}关系:")
            for record in result:
                print(f"{record['person']} 与 {record['company']} 有{rel_type}关系")
                results.append({
                    'person': record['person'],
                    'company': record['company']
                })
        
        elif query_type == 'node_relationships':
            # 查询特定节点的关系
            person_name = params.get('person_name', 'muyu')
            result = session.run("""
                MATCH (p:Person {name: $name})-[r]->(c)
                RETURN type(r) AS relationship, c.name AS connected_to, labels(c) AS node_type
            """, {'name': person_name})
            
            print(f"{person_name}的所有关系:")
            for record in result:
                print(f"关系类型: {record['relationship']}, 连接到: {record['connected_to']}, 节点类型: {record['node_type']}")
                results.append({
                    'relationship': record['relationship'],
                    'connected_to': record['connected_to'],
                    'node_type': record['node_type']
                })
              
        elif query_type == 'aggregation':
            # 聚合查询
            result = session.run("""
                MATCH (p:Person)-[:EMPLOYED_BY]->(c:Company)
                RETURN c.name AS company, count(p) AS employee_count, avg(p.age) AS avg_age
            """)
            
            print("公司员工统计:")
            for record in result:
                print(f"公司: {record['company']}, 员工数: {record['employee_count']}, 平均年龄: {round(record['avg_age'], 1)}")
                results.append({
                    'company': record['company'],
                    'employee_count': record['employee_count'],
                    'avg_age': record['avg_age']
                })
        
        elif query_type == 'group_by':
            # 条件分组查询
            result = session.run("""
                MATCH (p:Person)
                RETURN p.city AS city, count(p) AS person_count,
                       collect(p.name) AS names
                ORDER BY person_count DESC
            """)
            
            print("按城市分组的人员统计:")
            for record in result:
                print(f"城市: {record['city']}, 人数: {record['person_count']}, 姓名: {record['names']}")
                results.append({
                    'city': record['city'],
                    'person_count': record['person_count'],
                    'names': record['names']
                })
        
        
        elif query_type == 'complex_query':
            # 多条件复合查询
            min_age = params.get('min_age', 25)
            location = params.get('location', 'beijing')
            result = session.run("""
                MATCH (p:Person)-[r]->(c:Company)
                WHERE p.age > $min_age AND c.location = $location
                AND (type(r) = 'EMPLOYED_BY' OR type(r) = 'INVESTED_IN')
                RETURN p.name AS person, p.age AS age, 
                       type(r) AS relationship, c.name AS company
                ORDER BY p.age DESC
            """, {'min_age': min_age, 'location': location})
            
            print(f"{min_age}岁以上且与{location}公司有雇佣或投资关系的人:")
            for record in result:
                print(f"{record['person']} ({record['age']}岁) {record['relationship']} {record['company']}")
                results.append({
                    'person': record['person'],
                    'age': record['age'],
                    'relationship': record['relationship'],
                    'company': record['company']
                })
        
        elif query_type == 'param_query':
            # 参数化查询
            query_params = {
                'min_age': params.get('min_age', 25),
                'location': params.get('location', 'beijing'),
                'relationship_types': params.get('relationship_types', ["EMPLOYED_BY", "INVESTED_IN"])
            }
            
            result = session.run("""
                MATCH (p:Person)-[r]->(c:Company)
                WHERE p.age > $min_age AND c.location = $location
                AND type(r) IN $relationship_types
                RETURN p.name AS person, type(r) AS relationship, c.name AS company
            """, query_params)
            
            print(f"参数化查询结果 (年龄 > {query_params['min_age']}, 位置: {query_params['location']}):")
            for record in result:
                print(f"{record['person']} {record['relationship']} {record['company']}")
                results.append({
                    'person': record['person'],
                    'relationship': record['relationship'],
                    'company': record['company']
                })
        
        else:
            print(f"未知的查询类型: {query_type}")
            results.append({"error": f"未知的查询类型: {query_type}"})
    
    return results

In [147]:
from neo4j import GraphDatabase

NEO4J_URI="bolt://localhost"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="thy382324"  # 这里替换成自己设置的密码
NEO4J_DATABASE="neo4j"         # 社区版仅能使用默认的neo4j数据库，不支持创建其他数据库

driver = GraphDatabase.driver(
    NEO4J_URI, 
    auth=(NEO4J_USERNAME, NEO4J_PASSWORD)
    )

try:
    # 查询所有Person节点
    neo4j_query_examples(driver, 'all_persons')
    
    # 查询北京的人员
    neo4j_query_examples(driver, 'filter_by_city', {'city': 'beijing'})
    
    # 查询muyu的所有关系
    neo4j_query_examples(driver, 'node_relationships', {'person_name': 'muyu'})
    
    # 执行复杂查询
    neo4j_query_examples(driver, 'complex_query', {
        'min_age': 20,
        'location': 'beijing'
    })    
    
finally:
    # 关闭驱动
    driver.close()

所有Person节点:
姓名: muyu, 年龄: 32, 城市: beijing
姓名: zhangsan, 年龄: 23, 城市: shanghai
beijing的人员:
姓名: muyu, 年龄: 32
muyu的所有关系:
关系类型: EMPLOYED_BY, 连接到: Fufan, 节点类型: ['Company']
20岁以上且与beijing公司有雇佣或投资关系的人:
muyu (32岁) EMPLOYED_BY Fufan
zhangsan (23岁) EMPLOYED_BY Beyond


&emsp;&emsp;至此，我们已经完成了`Neo4j`的连接，并掌握了基础的`Cypher`语句，接下来，我们将介绍如何将`Microsoft GraphRAG`生成的各个`.parquet`文件导入到`Neo4j`的图数据库中，构建更加复杂和丰富的知识图谱。

# 4. Microsoft GraphRAG 导入

&emsp;&emsp;在上节课中，我们详细的介绍了`Microsoft GraphRAG`提供的两种方法将本地的非结构化数据构建知识图谱索引的完整流程及底层的实现原理，而无论是通过`Microsoft GraphRAG`的`CLI`工具，还是通过源码的`Poetry`, 其生成的文件都是一样的，即在`output`目录下会依次生成：

- documents.parquet
- text_units.parquet
- entities.parquet
- relationships.parquet
- communities.parquet
- community_reports.parquet

&emsp;&emsp;这些文件中存储了构建知识图谱索引的全部数据，而本节课我们将介绍如何将这些数据导入到`Neo4j`的图数据库中，并进行可视化。

## 4.1 构建批量导入函数

&emsp;&emsp;在开始逐步构建知识图谱之前，我们需要先构建一个批量的通用导入函数，用于将各个`.parquet`文件中的数据更高效的导入到`Neo4j`的图数据库中。 知识图谱存储的往往是非常大的数据量（数百万到数亿行的量级），有非常多的复杂关系，因此在实际的使用中，往往都是需要一些优化策略来做大规模数据的管理，常见的优化方案是：

1. 使用 `Neo4j Admin Import` 导入，这是最快的导入方式，可以比`Cypher`导入快`10-100`倍。
2. 并行处理与多线程。
3. 使用`APOC`批量导入。
4. 搭配其他优化策略，比如优化`Cypher`语句、优化数据格式、数据分区与分布式处理等。

&emsp;&emsp;在本期的`GraphRAG`系列课程中，我们会结合不同的场景依次介绍以上四种大规模数据导入的优化策略。本节课，我们首先介绍如何做并行处理与多线程。


&emsp;&emsp;在如下的`parallel_batched_import`函数中，实现的是一个使用并行批量导入数据到`Neo4j`的函数。它将大批量数据分成多个批次，然后使用线程池并行处理每个批次，从而提高导入的效果。

In [149]:
import time
import pandas as pd
import concurrent.futures
from neo4j import GraphDatabase


def parallel_batched_import(statement, df, batch_size=100, max_workers=8):
    """
    使用并行处理进行批量导入数据到Neo4j
    
    参数:
    - statement: Cypher查询语句，使用value作为每行数据的引用
    - df: 要导入的DataFrame
    - batch_size: 每批处理的行数
    - max_workers: 并行线程数
    
    返回:
    - 导入统计信息的字典
    """

    # 1. 初始化，计算总行数，批次数，并记录开始时间
    total = len(df)
    batches = (total + batch_size - 1) // batch_size  # 向上取整
    start_time = time.time()
    results = []
    
    print(f"开始并行导入 {total} 行数据，分为 {batches} 个批次，每批 {batch_size} 条")
    

    # 2. 定义批处理函数
    def process_batch(batch_idx):
        """
        批处理函数，用于处理每个批次的数据
        """

        # 1. 计算批次的起始和结束索引
        start = batch_idx * batch_size
        end = min(start + batch_size, total)
        batch = df.iloc[start:end]
        
        batch_start_time = time.time()
        
        try:
            with driver.session(database=NEO4J_DATABASE) as session:
                # NWIND 是 Cypher 查询语言中的一个关键字，用于将一个列表展开为多行。 $rows 是一个参数，表示将要传入的行数据
                # 完整意思是：将$rows参数（一个列表）中的每个元素展开，每个元素被赋值给变量value， 对每个value执行后续的Cypher语句
                #    id   name   age
                #    0    张三    25
                #    1    李四    30

                # 调用.to_dict("records")后，得到：
                # [
                #     {"id": 1, "name": "张三", "age": 25},
                #     {"id": 2, "name": "李四", "age": 30}
                # ]
                result = session.run(
                    "UNWIND $rows AS value " + statement,   
                    rows=batch.to_dict("records")
                )
                summary = result.consume()    # Neo4j 中用于处理查询结果的一个方法。它的主要作用是获取查询的摘要信息，包括执行统计、计数和其他相关信息
                batch_duration = time.time() - batch_start_time
                
                return {
                    "batch": batch_idx,
                    "rows": end - start,
                    "success": True,
                    "duration": batch_duration,
                    "counters": summary.counters   # summary.counters 是在执行 Cypher 查询后返回的统计信息
                }
        except Exception as e:
            batch_duration = time.time() - batch_start_time
            print(f"批次 {batch_idx} (行 {start}-{end-1}) 处理失败: {str(e)}")
            
            return {
                "batch": batch_idx,
                "rows": end - start,
                "success": False,
                "duration": batch_duration,
                "error": str(e)
            }
    
    # 使用线程池并行处理批次
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(process_batch, i) for i in range(batches)]
        
        # 处理完成的批次
        for i, future in enumerate(concurrent.futures.as_completed(futures)):
            result = future.result()
            results.append(result)
            
            if result["success"]:
                print(f"批次 {result['batch']} 完成: {result['rows']} 行, 耗时 {result['duration']:.2f}秒")
            else:
                print(f"批次 {result['batch']} 失败: {result['rows']} 行, 耗时 {result['duration']:.2f}秒, 错误: {result.get('error', '未知错误')}")
            
            # 显示进度
            print(f"进度: {i+1}/{batches} 批次完成 ({((i+1)/batches*100):.1f}%)")
    
    # 统计结果
    successful_rows = sum(r["rows"] for r in results if r["success"])
    failed_rows = sum(r["rows"] for r in results if not r["success"])
    
    duration = time.time() - start_time
    rows_per_second = successful_rows / duration if duration > 0 else 0
    
    print(f"导入完成: 总计 {total} 行, 成功 {successful_rows} 行, 失败 {failed_rows} 行")
    print(f"总耗时: {duration:.2f}秒, 平均速度: {rows_per_second:.2f} 行/秒")
    
    return {
        "total_rows": total,
        "successful_rows": successful_rows,
        "failed_rows": failed_rows,
        "duration_seconds": duration,
        "rows_per_second": rows_per_second,
        "batch_results": results
    }

&emsp;&emsp;理解了`parallel_batched_import`函数后，我们就依次将完成`Microsoft GraphRAG`索引过程得到的各个`.parquet`文件导入到`Neo4j`的图数据库中。

## 4.2 导入文档

&emsp;&emsp;`documents.parquet`文件中主要存储的是在执行索引之前，我们存放在`Input`文件夹下的原始的非结构化文档，比如`.txt`、`csv`等大批量文件。经过索引的构建流程后，会经过文档加载器解析出文档中的全部内容，并且会通过后处理，在进行`text_unit`的切分后，填充每个文档对应的`text_unit_ids`。 读取该文件的代码如下所示：

In [154]:
import pandas as pd
from tabulate import tabulate

df_documents = pd.read_parquet('D:\\agent\graphrag\graphrag\output\documents.parquet')  # 替换为实际路径

print(tabulate(df_documents, headers='keys', tablefmt='pretty', showindex=False, stralign='left', maxcolwidths=[20, 20, 20, 20, 20]))

+----------------------+-------------------+----------------------+----------------------+----------------------+---------------------------+----------+
| id                   | human_readable_id | title                | text                 | text_unit_ids        | creation_date             | metadata |
+----------------------+-------------------+----------------------+----------------------+----------------------+---------------------------+----------+
| a2c186a49102fbffaec5 | 1                 | source_2025-03-05.tx | 在过去的几十年中，全 | ['978b4116e7a59cdb80 | 2025-09-25 18:04:19 +0800 |          |
| c2453442cb5eb74fcdc6 |                   | t                    | 球科技行业经历了翻天 | 35ecc25d467c500cda95 |                           |          |
| d3fe298c03d72d30fbcd |                   |                      | 覆地的变化。以硅谷为 | eb850b8dbbb69f6d4716 |                           |          |
| 5cbddfbce438e2ee50eb |                   |                      | 中心的创新生态系统催 | 62ab9f49362a1b80d571 |         

<>:4: SyntaxWarning: invalid escape sequence '\g'
<>:4: SyntaxWarning: invalid escape sequence '\g'
C:\Users\Administrator\AppData\Local\Temp\ipykernel_23992\4016742172.py:4: SyntaxWarning: invalid escape sequence '\g'
  df_documents = pd.read_parquet('D:\\agent\graphrag\graphrag\output\documents.parquet')  # 替换为实际路径


&emsp;&emsp;其中各个字段的含义如下所示：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>Document DataFrame 字段解释</font></p>
<div class="center">

| 字段名              | 类型               | 说明                                                         |
|---------------------|--------------------|--------------------------------------------------------------|
| id                  | 字符串             | 唯一标识符，用于唯一标识每个文档（即单个文件）。                           |
| human_readable_id   | 字符串             | 可读的标识符，通常用于用户界面显示，便于用户理解和识别。   |
| title               | 字符串             | 加载的文档其对应的文件名。                     |
| text                | 字符串             | 通过文档解析器解析到的文档全部内容。                       |
| text_unit_ids       | 列表（字符串）     | 关联的文本单元的唯一标识符列表，文本单元是将文档切分的块。 |
| creation_date       | 日期时间           | 创建的日期和时间，以 UTC 或本地时间格式表示。 |
| metadata            | 字典（可选）       | 额外的元数据，包含与文档相关的其他信息，如作者、来源、标签等。此字段可以为空。 |

</div>

&emsp;&emsp;在知识图谱构建过程中，Document实体作为原始信息载体，其导入策略可以考虑以下原则：

- **节点标识**：需要使用`id`字段作为Document节点的唯一标识符，确保图谱中的文档实体一致性。
- **约束策略**：唯一性约束，确保文档的id是唯一的。
- **属性选择**：比如标题，文本内容等（我们上节课讲解了如何自定义属性（即通过 file_partten 和 file_filter 来定义出更多的列）），但是注意：text_unit_ids 有必要添加属性吗？ 可以做，但是很低效，因为后面可以通过关系来连接，从而降低查询的复杂度。举个例子：
    - 一个包含1000个文档，每个文档平均有50个文本单元的知识图谱：
        - 使用关系：创建50,000个CONTAINS关系，每个关系指向一个TextUnit节点
        - 使用属性：每个Document节点包含一个平均有50个ID的列表，需要在查询时进行列表处理
- **标签策略**：基础标签，为所有文档添加`__Document__`标签。

&emsp;&emsp;因此，把上述的策略转化成代码。就如下所示：

In [152]:
# 创建Document节点
def create_document_nodes(df_documents):
    # 首先创建唯一性约束
    with driver.session(database=NEO4J_DATABASE) as session:
        try:
            session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (d:__Document__) REQUIRE d.id IS UNIQUE")
        except Exception as e:
            print(f"创建约束时出错 (可能已存在): {e}")
    
    # 导入文档
    # MERGE与ON CREATE SET组合，是一种条件性属性设置方式：特点是：
    # 1. 只在节点首次创建时设置属性
    # 2. 如果节点已存在，不会修改现有属性
    # 适合初始导入场景，避免覆盖已有数据
    cypher_statement = """
    MERGE (d:__Document__ {id: value.id})
    ON CREATE SET 
        d.human_readable_id = value.human_readable_id,
        d.title = value.title,
        d.text = value.text,
        d.creation_date = value.creation_date,
        d.import_timestamp = timestamp()
    """
    
    return parallel_batched_import(cypher_statement, df_documents)

&emsp;&emsp;接下来，连接到`Neo4j`数据库，并调用`create_document_nodes`函数，导入文档。代码如下：

In [153]:
NEO4J_URI="bolt://localhost"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="thy382324"
NEO4J_DATABASE="neo4j"

driver = GraphDatabase.driver(
    NEO4J_URI, 
    auth=(NEO4J_USERNAME, NEO4J_PASSWORD)
    )

# 执行 document.parquet 文件的导入
create_document_nodes(df_documents)

开始并行导入 4 行数据，分为 1 个批次，每批 100 条
批次 0 完成: 4 行, 耗时 0.37秒
进度: 1/1 批次完成 (100.0%)
导入完成: 总计 4 行, 成功 4 行, 失败 0 行
总耗时: 0.39秒, 平均速度: 10.32 行/秒


{'total_rows': 4,
 'successful_rows': 4,
 'failed_rows': 0,
 'duration_seconds': 0.38764476776123047,
 'rows_per_second': 10.318725628882465,
 'batch_results': [{'batch': 0,
   'rows': 4,
   'success': True,
   'duration': 0.374652624130249,
   'counters': {'_contains_updates': True, 'labels_added': 3, 'nodes_created': 3, 'properties_set': 18}}]}

&emsp;&emsp;看到创建完成后，在`localhost:7474`的`Neo4j`浏览器中，可以看到`__Document__`标签的节点已经创建完成。如下图所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503121817980.png" width=80%></div>

&emsp;&emsp;至此，我们已经完成了`documents.parquet`文件的导入。接下来执行`text_units.parquet`文件的导入。

## 4.3 导入文本单元

&emsp;&emsp;`text_units.parquet`文件中主要存储的是内容是根据`settings.yaml`文件中定义的切分策略，将`documents.parquet`文件中的内容切分出来的文本单元。同时会通过后处理，关联到经过实体和关系提取`Workflow`后的实体和关系。读取该文件的代码如下所示：

In [156]:
from tabulate import tabulate

df_text_units = pd.read_parquet('D:\\agent\graphrag\graphrag\output\\text_units.parquet')  # 替换为实际路径
# 假设 df 是你的 DataFrame
print(tabulate(df_text_units, headers='keys', tablefmt='pretty', showindex=False, stralign='left', maxcolwidths=[20, 20, 20, 20, 20]))

+----------------------+-------------------+----------------------+----------+----------------------+------------------------------------------+------------------------------------------+---------------+
| id                   | human_readable_id | text                 | n_tokens | document_ids         | entity_ids                               | relationship_ids                         | covariate_ids |
+----------------------+-------------------+----------------------+----------+----------------------+------------------------------------------+------------------------------------------+---------------+
| 287e26180757e0d0d78a | 1                 | 测试3                | 2        | ['585d63a2b9b6ac1f6d |                                          |                                          | []            |
| afdd1cd50c6a4118bd89 |                   |                      |          | 5ae1ad363737b0b9baf0 |                                          |                                          

<>:3: SyntaxWarning: invalid escape sequence '\g'
<>:3: SyntaxWarning: invalid escape sequence '\g'
C:\Users\Administrator\AppData\Local\Temp\ipykernel_23992\1545793398.py:3: SyntaxWarning: invalid escape sequence '\g'
  df_text_units = pd.read_parquet('D:\\agent\graphrag\graphrag\output\\text_units.parquet')  # 替换为实际路径


<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>text_units DataFrame 字段解释</font></p>
<div class="center">

| 字段名              | 类型               | 说明                                                         |
|---------------------|--------------------|--------------------------------------------------------------|
| id                  | 字符串             | 唯一标识符，用于唯一标识每个文档块（即从单个文档中切分出来的完整块）。                          |
| human_readable_id   | 字符串             | 可读的标识符，通常用于用户界面显示，便于用户理解和识别。   |
| text                | 字符串             | 该文本块中的内容                       |
| n_tokens            | 整数               | 按照切分策略切分出来的最终文本长度。                         |
| document_ids        | 列表（字符串）     | 关联的文档的唯一标识符列表，用于将文本块与document中的文档id关联。   |
| entity_ids          | 列表（字符串）     | 关联的实体的唯一标识符列表，表示该文本块中都提取出来了哪些实体。       |
| relationship_ids    | 列表（字符串）     | 关联的关系的唯一标识符列表，表示该文本块中都提取出来了哪些关系。       |
| covariate_ids       | 列表（字符串）     | 关联的协变量的唯一标识符列表，用于统计分析或建模。   |
</div>

&emsp;&emsp;在知识图谱构建过程中，TextUnit实体作为文本切分后的产物，其导入策略可以考虑以下原则：

- **节点标识**：需要使用`id`字段作为TextUnit节点的唯一标识符，确保图谱中文档块实体一致性。
- **约束策略**：唯一性约束，确保文本块的id是唯一的。
- **属性选择**：比如文本内容，文本长度等，根据需求选择。
- **关系策略**：创建`PART_OF`关系，将TextUnit节点与Document节点关联起来，表示该文本块属于哪个文档。

&emsp;&emsp;因此，把上述的策略转化成代码。就如下所示：

In [158]:
def setup_chunk_constraints():
    """创建Chunk标签的约束"""
    with driver.session(database=NEO4J_DATABASE) as session:
        try:
            # 创建Chunk.id唯一性约束
            session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (c:__Chunk__) REQUIRE c.id IS UNIQUE")
            print("已创建Chunk.id唯一性约束")
        except Exception as e:
            print(f"创建__Chunk__约束时出错 (可能已存在): {e}")
            # 尝试旧版本Neo4j的语法
            try:
                session.run("CREATE CONSTRAINT ON (c:__Chunk__) ASSERT c.id IS UNIQUE")
                print("已使用旧语法创建__Chunk__.id唯一性约束")
            except Exception as e2:
                print(f"使用旧语法创建约束也失败: {e2}")

In [159]:
def import_chunks(df_chunks, batch_size=100, max_workers=8):
    """
    导入文档块(Chunk)到Neo4j
    
    参数:
    - df_chunks: 包含文档块数据的DataFrame
    - batch_size: 每批处理的行数
    - max_workers: 并行线程数
    
    返回:
    - 导入统计信息的字典
    """
    # 1. 创建Chunk的约束
    setup_chunk_constraints()
    
    # 2. 创建Chunk节点
    print("开始导入Chunk节点...")
    chunk_statement = """
    MERGE (c:__Chunk__ {id: value.id})
    SET c.text = value.text,
        c.n_tokens = value.n_tokens,
        c.human_readable_id = value.human_readable_id,
        // 设置显示名称为human_readable_id
        c.name = value.human_readable_id
    """
    
    chunk_result = parallel_batched_import(chunk_statement, df_chunks, batch_size, max_workers)
    
    # 3. 准备关系数据 - 正确处理嵌套的document_ids并过滤无效数据
    print("准备Chunk-Document关系数据...")
    relations_data = []
    
    for idx, row in df_chunks.iterrows():
        chunk_id = row['id']
        doc_ids_container = row['document_ids']
        
        # 处理嵌套结构
        flat_doc_ids = []
        
        # 如果是列表，展平它
        if isinstance(doc_ids_container, list):
            for item in doc_ids_container:
                # 如果是NumPy数组
                if hasattr(item, 'dtype') and hasattr(item, 'tolist'):
                    flat_doc_ids.extend(item.tolist())
                # 如果是列表
                elif isinstance(item, list):
                    flat_doc_ids.extend(item)
                # 如果是单个值
                else:
                    flat_doc_ids.append(item)
        # 如果不是列表，直接添加
        elif doc_ids_container is not None:
            flat_doc_ids.append(doc_ids_container)
        
        # 记录处理后的document_ids
        if idx < 5:  # 只记录前几行，避免日志过大
            print(f"行 {idx} 处理后的document_ids: {flat_doc_ids}")
        
        # 为每个文档ID创建关系，过滤掉无效的ID
        for doc_id in flat_doc_ids:
            if doc_id is not None and str(doc_id).strip() != '':
                # 确保doc_id是字符串并过滤掉特殊格式
                doc_id_str = str(doc_id).strip()
                
                # 过滤掉不符合预期格式的ID
                if doc_id_str.startswith('<elementId>') or doc_id_str.startswith('<id>'):
                    print(f"跳过无效的document_id: {doc_id_str}")
                    continue
                
                # 如果是列表形式的字符串，提取实际ID
                if doc_id_str.startswith('[') and doc_id_str.endswith(']'):
                    try:
                        # 尝试解析列表字符串
                        import ast
                        id_list = ast.literal_eval(doc_id_str)
                        if isinstance(id_list, list) and len(id_list) > 0:
                            doc_id_str = str(id_list[0])
                    except:
                        print(f"无法解析列表格式的document_id: {doc_id_str}")
                        continue
                
                relations_data.append({
                    'chunk_id': chunk_id,
                    'document_id': doc_id_str
                })
    
    # 4. 确保Document节点存在
    if relations_data:
        unique_doc_ids = set(item['document_id'] for item in relations_data)
        print(f"发现 {len(unique_doc_ids)} 个唯一的document_id")
        
        # 创建Document节点
        docs_df = pd.DataFrame({'id': list(unique_doc_ids)})
        document_statement = """
        MERGE (d:__Document__ {id: value.id})
        // 设置显示名称
        SET d.name = value.id
        """
        doc_result = parallel_batched_import(document_statement, docs_df, batch_size, max_workers)
        print(f"已创建 {doc_result['successful_rows']} 个Document节点")
        
        # 5. 创建关系
        print(f"开始创建 {len(relations_data)} 个Chunk-Document关系...")
        df_relations = pd.DataFrame(relations_data)
        
        relation_statement = """
        MATCH (c:__Chunk__ {id: value.chunk_id})
        MATCH (d:__Document__ {id: value.document_id})
        MERGE (c)-[:PART_OF]->(d)
        """
        
        relation_result = parallel_batched_import(relation_statement, df_relations, batch_size, max_workers)
        print(f"已创建 {relation_result['successful_rows']} 个Chunk-Document关系")
    else:
        print("没有找到有效的Chunk-Document关系数据")
    
    # 6. 验证结果
    with driver.session(database=NEO4J_DATABASE) as session:
        # 检查Chunk节点数量
        result = session.run("MATCH (c:__Chunk__) RETURN count(c) as count")
        chunk_count = result.single()["count"]
        
        # 检查Document节点数量
        result = session.run("MATCH (d:__Document__) RETURN count(d) as count")
        doc_count = result.single()["count"]
        
        # 检查PART_OF关系数量
        result = session.run("MATCH ()-[r:PART_OF]->() RETURN count(r) as count")
        relation_count = result.single()["count"]
        
        print(f"验证结果: {chunk_count} 个Chunk节点, {doc_count} 个Document节点, {relation_count} 个PART_OF关系")
    
    return chunk_result

&emsp;&emsp;接下来，连接到`Neo4j`数据库，并调用`import_chunks`函数，导入文本单元。代码如下：

In [160]:
NEO4J_URI="bolt://localhost"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="thy382324"
NEO4J_DATABASE="neo4j"

driver = GraphDatabase.driver(
    NEO4J_URI, 
    auth=(NEO4J_USERNAME, NEO4J_PASSWORD)
    )

# 执行 text_units.parquet 文件的导入
import_chunks(df_text_units)

已创建Chunk.id唯一性约束
开始导入Chunk节点...
开始并行导入 6 行数据，分为 1 个批次，每批 100 条
批次 0 完成: 6 行, 耗时 0.16秒
进度: 1/1 批次完成 (100.0%)
导入完成: 总计 6 行, 成功 6 行, 失败 0 行
总耗时: 0.16秒, 平均速度: 36.59 行/秒
准备Chunk-Document关系数据...
行 0 处理后的document_ids: [array(['585d63a2b9b6ac1f6d5ae1ad363737b0b9baf041a8a9b6e2ed4e24d5453f643e4ebdc5db99c67d37435308778d1cfd820524e065475b0875c71921bdd85d6836'],
      dtype=object)]
行 1 处理后的document_ids: [array(['a2c186a49102fbffaec5c2453442cb5eb74fcdc6d3fe298c03d72d30fbcd5cbddfbce438e2ee50eb802066c35030e8a63b1113f0bc70ebdcc0c4a21d5a055355'],
      dtype=object)]
行 2 处理后的document_ids: [array(['a2c186a49102fbffaec5c2453442cb5eb74fcdc6d3fe298c03d72d30fbcd5cbddfbce438e2ee50eb802066c35030e8a63b1113f0bc70ebdcc0c4a21d5a055355',
       'a2c186a49102fbffaec5c2453442cb5eb74fcdc6d3fe298c03d72d30fbcd5cbddfbce438e2ee50eb802066c35030e8a63b1113f0bc70ebdcc0c4a21d5a055355'],
      dtype=object)]
行 3 处理后的document_ids: [array(['a2c186a49102fbffaec5c2453442cb5eb74fcdc6d3fe298c03d72d30fbcd5cbddfbce438e2ee50eb802066c35

{'total_rows': 6,
 'successful_rows': 6,
 'failed_rows': 0,
 'duration_seconds': 0.1640005111694336,
 'rows_per_second': 36.58525182157042,
 'batch_results': [{'batch': 0,
   'rows': 6,
   'success': True,
   'duration': 0.1609959602355957,
   'counters': {'_contains_updates': True, 'labels_added': 6, 'nodes_created': 6, 'properties_set': 30}}]}

&emsp;&emsp;同样第，执行完成后可以在`localhost:7474`的`Neo4j`浏览器中，可以看到`__Chunk__`标签的节点已经创建完成，以及`__Chunk__`与`__Document__`之间的关系也已经创建完成。如下图所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503121844987.png" width=80%></div>

&emsp;&emsp;至此，我们就完成了`text_units.parquet`文件的导入。接下来，我们开始导入`entities.parquet`文件。

## 4.4 导入实体

&emsp;&emsp;`entities.parquet`文件中主要存储的通过提示词，借助大模型从`text_units`中提取出来的实体。读取该文件的代码如下所示：

In [161]:
# 使用绝对路径读取 Parquet 文件
df_entities = pd.read_parquet('D:\\agent\graphrag\graphrag\output\entities.parquet')  # 替换为实际路径

# 假设 df 是你的 DataFrame
print(tabulate(df_entities, headers='keys', tablefmt='pretty', showindex=False, stralign='left', maxcolwidths=[20, 20, 20, 20, 20]))

+----------------------+-------------------+----------------------+--------------+----------------------+--------------------------------------------------------------------------------------------------------------------------------------+-----------+--------+---+---+
| id                   | human_readable_id | title                | type         | description          | text_unit_ids                                                                                                                        | frequency | degree | x | y |
+----------------------+-------------------+----------------------+--------------+----------------------+--------------------------------------------------------------------------------------------------------------------------------------+-----------+--------+---+---+
| b87d649f-388a-4766-8 | 0                 | APPLE INC.           | ORGANIZATION | Apple Inc. is a      | ['978b4116e7a59cdb8035ecc25d467c500cda95eb850b8dbbb69f6d471662ab9f49362a1b80d5715d8f

<>:2: SyntaxWarning: invalid escape sequence '\g'
<>:2: SyntaxWarning: invalid escape sequence '\g'
C:\Users\Administrator\AppData\Local\Temp\ipykernel_23992\3793087927.py:2: SyntaxWarning: invalid escape sequence '\g'
  df_entities = pd.read_parquet('D:\\agent\graphrag\graphrag\output\entities.parquet')  # 替换为实际路径


&emsp;&emsp;各个字段的解释如下所示：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>entities DataFrame 字段解释</font></p>
<div class="center">

| 字段名              | 类型               | 说明                                                         |
|---------------------|--------------------|--------------------------------------------------------------|
| id                  | 字符串             | 唯一标识符，用于唯一标识每个实体记录。                           |
| human_readable_id   | 字符串             | 可读的标识符，用于用户界面显示，便于用户理解和识别。   |
| title               | 字符串             | 实体的名称或主题。                   |
| type                | 字符串             | 实体的类型，例如 `GEO` 表示地理位置。                       |
| description         | 字符串             | 对实体的详细描述，提供更多上下文信息。                     |
| text_unit_ids       | 列表（字符串）     | 关联的文本单元的唯一标识符列表，用于将实体与文本块关联。 |
| frequency           | 整数               | 实体出现的频率，表示该实体在数据中的重要性或常见程度。     |
| degree              | 整数               | 实体的度数，通常表示与其他实体的连接数量。                 |
| x                   | 浮点数             | 实体在二维空间中的 x 坐标，通常用于可视化。                |
| y                   | 浮点数             | 实体在二维空间中的 y 坐标，通常用于可视化。                |

</div>

&emsp;&emsp;在知识图谱构建过程中，其导入策略可以考虑以下原则：

- **节点标识**：需要使用`id`字段作为Entity节点的唯一标识符，确保图谱中实体节点一致性。同时为了避免重复，`name`字段也需要作为Entity节点的唯一标识符。
- **约束策略**：唯一性约束，确保实体的id是唯一的。
- **属性选择**：比如实体的名称，描述，频率，度数等，根据需求选择。
- **标签策略**：给节点添加动态标签（根据 实体的 type 添加）
- **关系策略**：创建`HAS_ENTITY`关系，将Entity节点与Chunk节点关联起来，表示该实体属于哪个文本块。

&emsp;&emsp;因此，把上述的策略转化成代码。就如下所示：

In [162]:
def setup_entity_constraints():
    """创建Entity标签的约束"""
    with driver.session(database=NEO4J_DATABASE) as session:
        try:
            # 创建Entity.id唯一性约束
            session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (e:__Entity__) REQUIRE e.id IS UNIQUE")
            session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (e:__Entity__) REQUIRE e.name IS UNIQUE")
            print("已创建__Entity__.id唯一性约束")
        except Exception as e:
            print(f"创建__Entity__约束时出错 (可能已存在): {e}")
            # 尝试旧版本Neo4j的语法
            try:
                session.run("CREATE CONSTRAINT ON (e:__Entity__) ASSERT e.id IS UNIQUE")
                print("已使用旧语法创建__Entity__.id唯一性约束")
            except Exception as e2:
                print(f"使用旧语法创建约束也失败: {e2}")

def import_entities(df_entities, batch_size=100, max_workers=8):
    """
    导入实体(Entity)到Neo4j
    
    参数:
    - df_entities: 包含实体数据的DataFrame
    - batch_size: 每批处理的行数
    - max_workers: 并行线程数
    
    返回:
    - 导入统计信息的字典
    """
    # 1. 创建Entity的约束
    setup_entity_constraints()
    
    # 2. 预处理text_unit_ids - 确保是列表格式
    print("预处理text_unit_ids...")
    
    # 创建DataFrame的副本
    df_entities = df_entities.copy()
    
    # 处理text_unit_ids字段
    for idx, row in df_entities.iterrows():
        text_unit_ids = row.get('text_unit_ids')
        
        # 如果不是列表，转换为列表
        if not isinstance(text_unit_ids, list):
            if isinstance(text_unit_ids, str):
                try:
                    # 尝试解析JSON字符串
                    import json
                    text_unit_ids = json.loads(text_unit_ids)
                except:
                    # 如果解析失败，将其作为单个元素的列表
                    text_unit_ids = [text_unit_ids]
            elif hasattr(text_unit_ids, 'dtype') and hasattr(text_unit_ids, 'tolist'):
                # 处理NumPy数组
                text_unit_ids = text_unit_ids.tolist()
            else:
                # 其他类型，转为列表
                text_unit_ids = [text_unit_ids] if text_unit_ids is not None else []
        
        # 处理嵌套列表
        flat_text_unit_ids = []
        for item in text_unit_ids:
            if isinstance(item, list) or (hasattr(item, 'dtype') and hasattr(item, 'tolist')):
                if hasattr(item, 'tolist'):
                    flat_text_unit_ids.extend(item.tolist())
                else:
                    flat_text_unit_ids.extend(item)
            else:
                flat_text_unit_ids.append(item)
        
        # 确保所有ID都是字符串且非空
        flat_text_unit_ids = [str(id) for id in flat_text_unit_ids if id is not None and str(id).strip() != '']
        
        # 更新DataFrame
        df_entities.at[idx, 'text_unit_ids'] = flat_text_unit_ids
    
    # 3. 检查Neo4j功能支持
    print("检查Neo4j功能支持...")
    has_apoc = False
    has_vector = False
    
    try:
        with driver.session(database=NEO4J_DATABASE) as session:
            # 检查APOC
            try:
                result = session.run("RETURN apoc.version() AS version")
                version = result.single()["version"]
                has_apoc = True
                print(f"APOC插件已安装，版本: {version}")
            except Exception as e:
                print(f"检查APOC插件时出错 (可能未安装): {e}")
            
    except Exception as e:
        print(f"检查Neo4j功能支持时出错: {e}")
    
    # 4. 导入Entity节点并创建关系
    print("开始导入__Entity__节点并创建关系...")
    
    # 根据功能支持构建Cypher语句
    if has_apoc and has_vector:
        # 完整功能支持
        entity_statement = """
        MERGE (e:__Entity__ {id:value.id})
        SET e += value {.human_readable_id, .description, .frequency, .degree, .x, .y}
        SET e.name = replace(coalesce(value.title, value.human_readable_id, ''), '"', '')
        
        WITH e, value
        CALL db.create.setNodeVectorProperty(e, "description_embedding", value.description_embedding)
        
        WITH e, value
        CALL apoc.create.addLabels(e, 
            CASE WHEN coalesce(value.type,"") = "" 
            THEN [] 
            ELSE [apoc.text.upperCamelCase(replace(value.type,'"',''))] 
            END
        ) YIELD node
        
        WITH node as e, value
        UNWIND value.text_unit_ids AS text_unit
        MATCH (c:__Chunk__ {id:text_unit})
        MERGE (c)-[:HAS_ENTITY]->(e)
        """
    elif has_apoc:
        # 只有APOC支持，没有向量支持
        entity_statement = """
        MERGE (e:__Entity__ {id:value.id})
        SET e += value {.human_readable_id, .description, .frequency, .degree, .x, .y}
        SET e.name = replace(coalesce(value.title, value.human_readable_id, ''), '"', '')
        
        WITH e, value
        CALL apoc.create.addLabels(e, 
            CASE WHEN coalesce(value.type,"") = "" 
            THEN [] 
            ELSE [apoc.text.upperCamelCase(replace(value.type,'"',''))] 
            END
        ) YIELD node
        
        WITH node as e, value
        UNWIND value.text_unit_ids AS text_unit
        MATCH (c:__Chunk__ {id:text_unit})
        MERGE (c)-[:HAS_ENTITY]->(e)
        """
    elif has_vector:
        # 只有向量支持，没有APOC支持
        entity_statement = """
        MERGE (e:__Entity__ {id:value.id})
        SET e += value {.human_readable_id, .description, .frequency, .degree, .x, .y}
        SET e.name = replace(coalesce(value.title, value.human_readable_id, ''), '"', '')
        
        WITH e, value
        CALL db.create.setNodeVectorProperty(e, "description_embedding", value.description_embedding)
        
        WITH e, value
        UNWIND value.text_unit_ids AS text_unit
        MATCH (c:__Chunk__ {id:text_unit})
        MERGE (c)-[:HAS_ENTITY]->(e)
        """
    else:
        # 基本功能，无APOC和向量支持
        entity_statement = """
        MERGE (e:__Entity__ {id:value.id})
        SET e += value {.human_readable_id, .description, .frequency, .degree, .x, .y}
        SET e.name = replace(coalesce(value.title, value.human_readable_id, ''), '"', '')
        
        WITH e, value
        UNWIND value.text_unit_ids AS text_unit
        MATCH (c:__Chunk__ {id:text_unit})
        MERGE (c)-[:HAS_ENTITY]->(e)
        """
    
    # 执行导入
    entity_result = parallel_batched_import(entity_statement, df_entities, batch_size, max_workers)
    
    # 5. 验证结果
    with driver.session(database=NEO4J_DATABASE) as session:
        # 检查Entity节点数量
        result = session.run("MATCH (e:__Entity__) RETURN count(e) as count")
        entity_count = result.single()["count"]
        
        # 检查HAS_ENTITY关系数量
        result = session.run("MATCH (c:__Chunk__)-[r:HAS_ENTITY]->(e:__Entity__) RETURN count(r) as count")
        relation_count = result.single()["count"]
        
        # 检查动态标签
        result = session.run("CALL db.labels() YIELD label WHERE label <> '__Entity__' AND label <> '__Chunk__' AND label <> '__Document__' RETURN collect(label) as labels")
        dynamic_labels = result.single()["labels"]
        
        print(f"验证结果: {entity_count} 个__Entity__节点, {relation_count} 个HAS_ENTITY关系")
        print(f"动态标签: {dynamic_labels}")
    
    return entity_result

&emsp;&emsp;如果需要安装`apoc`插件，可以进入该地址：https://github.com/neo4j/apoc/releases， 下载对应的版本，然后将其放置在`Neo4j`的`plugins`文件夹下，重启`Neo4j`即可。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503131146422.png" width=80%></div>

In [163]:
NEO4J_URI="bolt://localhost"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="thy382324"
NEO4J_DATABASE="neo4j"

driver = GraphDatabase.driver(
    NEO4J_URI, 
    auth=(NEO4J_USERNAME, NEO4J_PASSWORD)
    )


# 导入实体
import_entities(df_entities, batch_size=100, max_workers=8)

已创建__Entity__.id唯一性约束
预处理text_unit_ids...
检查Neo4j功能支持...
APOC插件已安装，版本: 2025.02.0
开始导入__Entity__节点并创建关系...
开始并行导入 45 行数据，分为 1 个批次，每批 100 条
批次 0 完成: 45 行, 耗时 0.55秒
进度: 1/1 批次完成 (100.0%)
导入完成: 总计 45 行, 成功 45 行, 失败 0 行
总耗时: 0.56秒, 平均速度: 80.79 行/秒
验证结果: 45 个__Entity__节点, 86 个HAS_ENTITY关系
动态标签: ['Person', 'Organization', 'Geo', 'Event', '__Relationship__', '__Community__']


{'total_rows': 45,
 'successful_rows': 45,
 'failed_rows': 0,
 'duration_seconds': 0.5570182800292969,
 'rows_per_second': 80.78729480410084,
 'batch_results': [{'batch': 0,
   'rows': 45,
   'success': True,
   'duration': 0.5540056228637695,
   'counters': {'_contains_updates': True, 'labels_added': 45, 'relationships_created': 86, 'nodes_created': 45, 'properties_set': 360}}]}

&emsp;&emsp;执行完成后，可以在`localhost:7474`的`Neo4j`浏览器中，可以看到会根据`type`字段，创建了不同的标签。同时，`__Entity__`节点与`__Chunk__`节点之间，也创建了`HAS_ENTITY`关系。如下图所示：


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503121911563.png" width=80%></div>

&emsp;&emsp;至此，我们就完成了`entities.parquet`文件的导入。接下来，我们开始导入`relationships.parquet`文件。

## 4.5 导入关系

&emsp;&emsp;在`relationships.parquet`文件中，主要存储的是通过提示词，借助大模型从`text_units`中提取出来的关系。读取该文件的代码如下所示：

In [164]:
# 使用绝对路径读取 Parquet 文件
df_relations = pd.read_parquet('D:\\agent\graphrag\graphrag\output\\relationships.parquet')  # 替换为实际路径

# 假设 df 是你的 DataFrame
print(tabulate(df_relations, headers='keys', tablefmt='pretty', showindex=False, stralign='left', maxcolwidths=[20, 20, 20, 20, 20]))

+----------------------+-------------------+-------------------+----------------------+----------------------+--------+-----------------+--------------------------------------------------------------------------------------------------------------------------------------+
| id                   | human_readable_id | source            | target               | description          | weight | combined_degree | text_unit_ids                                                                                                                        |
+----------------------+-------------------+-------------------+----------------------+----------------------+--------+-----------------+--------------------------------------------------------------------------------------------------------------------------------------+
| 5faae0e7-1447-4869-a | 0                 | APPLE INC.        | STEVE JOBS           | Steve Jobs was a co- | 10.0   | 15              | ['978b4116e7a59cdb8035ecc25d467c500cda95eb8

<>:2: SyntaxWarning: invalid escape sequence '\g'
<>:2: SyntaxWarning: invalid escape sequence '\g'
C:\Users\Administrator\AppData\Local\Temp\ipykernel_23992\4002986165.py:2: SyntaxWarning: invalid escape sequence '\g'
  df_relations = pd.read_parquet('D:\\agent\graphrag\graphrag\output\\relationships.parquet')  # 替换为实际路径


&emsp;&emsp;各个字段的解释如下所示：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>relationships DataFrame 字段解释</font></p>
<div class="center">

| 字段名              | 类型               | 说明                                                         |
|---------------------|--------------------|--------------------------------------------------------------|
| id                  | 字符串             | 唯一标识符，用于唯一标识每个关系记录。                       |
| human_readable_id   | 字符串             | 可读的标识符，通常用于用户界面显示，便于用户理解和识别。   |
| source              | 字符串             | 关系的源实体，表示关系的起始节点。                         |
| target              | 字符串             | 关系的目标实体，表示关系的结束节点。                       |
| description         | 字符串             | 对关系的详细描述，提供更多上下文信息。                     |
| weight              | 浮点数             | 关系的权重，通常用于表示关系的重要性或强度。               |
| combined_degree     | 整数               | 关系的组合度，表示与其他实体的连接数量。                   |
| text_unit_ids       | 列表（字符串）     | 关联的文本单元的唯一标识符列表，通常用于将关系与文本关联。 |

</div>

&emsp;&emsp;在知识图谱构建过程中，我们可以使用已经创建好的`__Entity__`节点，然后根据`source`和`target`字段，创建关系。其导入策略可以考虑以下原则：

- **节点标识**：需要使用`id`字段作为Relationship节点的唯一标识符，确保图谱中关系节点一致性。
- **约束策略**：唯一性约束，确保关系的id是唯一的。
- **属性选择**：比如关系的描述，权重，组合度等，根据需求选择。
- **关系策略**：查找两个实体节点（source 和 target），创建或更新这两个实体之间的 RELATED 关系。

&emsp;&emsp;因此，把上述的策略转化成代码。就如下所示：其中比较关键的是：

- RELATED关系：连接两个__Entity__节点，表示实体之间的直接语义关系，例如：(Person:Entity)-[:RELATED]->(Company:Entity)
- HAS_RELATIONSHIP关系：连接__Chunk__节点和__Relationship__节点，表示某个文本块中提到了某种关系，例如：(TextChunk:Chunk)-[:HAS_RELATIONSHIP]->(EntityRelationship:Relationship)

In [165]:
def setup_relationship_constraints():
    """创建Relationship标签的约束"""
    with driver.session(database=NEO4J_DATABASE) as session:
        try:
            # 创建Relationship.id唯一性约束
            session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (r:__Relationship__) REQUIRE r.id IS UNIQUE")
            print("已创建__Relationship__.id唯一性约束")
        except Exception as e:
            print(f"创建__Relationship__约束时出错 (可能已存在): {e}")
            # 尝试旧版本Neo4j的语法
            try:
                session.run("CREATE CONSTRAINT ON (r:__Relationship__) ASSERT r.id IS UNIQUE")
                print("已使用旧语法创建__Relationship__.id唯一性约束")
            except Exception as e2:
                print(f"使用旧语法创建约束也失败: {e2}")

def import_relationships(df_relationships, batch_size=100, max_workers=8):
    """
    导入关系数据到Neo4j
    
    参数:
    - df_relationships: 包含关系数据的DataFrame
    - batch_size: 每批处理的行数
    - max_workers: 并行线程数
    
    返回:
    - 导入统计信息的字典
    """
    # 1. 创建Relationship的约束
    setup_relationship_constraints()
    
    # 2. 检查标签和节点是否存在
    with driver.session(database=NEO4J_DATABASE) as session:
        # 检查Entity和Chunk节点数量
        result = session.run("MATCH (e:__Entity__) RETURN count(e) as count")
        entity_count = result.single()["count"]
        
        result = session.run("MATCH (c:__Chunk__) RETURN count(c) as count")
        chunk_count = result.single()["count"]
        
        print(f"节点数量: {entity_count} 个__Entity__节点, {chunk_count} 个__Chunk__节点")
        
        if entity_count == 0:
            print("没有__Entity__节点，请先导入Entity数据")
        
        if chunk_count == 0:
            print("没有__Chunk__节点，请先导入Chunk数据")
    
    # 3. 预处理text_unit_ids - 确保是列表格式
    print("预处理text_unit_ids...")
    
    # 创建DataFrame的副本
    df_relationships = df_relationships.copy()
    
    # 处理text_unit_ids字段
    for idx, row in df_relationships.iterrows():
        text_unit_ids = row.get('text_unit_ids')
        
        # 如果不是列表，转换为列表
        if not isinstance(text_unit_ids, list):
            if isinstance(text_unit_ids, str):
                try:
                    # 尝试解析JSON字符串
                    import json
                    text_unit_ids = json.loads(text_unit_ids)
                except:
                    # 如果解析失败，将其作为单个元素的列表
                    text_unit_ids = [text_unit_ids]
            elif hasattr(text_unit_ids, 'dtype') and hasattr(text_unit_ids, 'tolist'):
                # 处理NumPy数组
                text_unit_ids = text_unit_ids.tolist()
            else:
                # 其他类型，转为列表
                text_unit_ids = [text_unit_ids] if text_unit_ids is not None else []
        
        # 处理嵌套列表
        flat_text_unit_ids = []
        for item in text_unit_ids:
            if isinstance(item, list) or (hasattr(item, 'dtype') and hasattr(item, 'tolist')):
                if hasattr(item, 'tolist'):
                    flat_text_unit_ids.extend(item.tolist())
                else:
                    flat_text_unit_ids.extend(item)
            else:
                flat_text_unit_ids.append(item)
        
        # 确保所有ID都是字符串且非空
        flat_text_unit_ids = [str(id) for id in flat_text_unit_ids if id is not None and str(id).strip() != '']
        
        # 更新DataFrame
        df_relationships.at[idx, 'text_unit_ids'] = flat_text_unit_ids
    
    # 4. 验证source和target是否存在
    print("验证source和target实体...")
    sample_rows = df_relationships.head(min(5, len(df_relationships)))  # 取前5行进行验证
    
    with driver.session(database=NEO4J_DATABASE) as session:
        for _, row in sample_rows.iterrows():
            source_id = row['source']
            target_id = row['target']
            
            # 检查source实体
            result = session.run("MATCH (e:__Entity__ {id: $id}) RETURN count(e) as count", id=source_id)
            source_exists = result.single()["count"] > 0
            
            # 检查target实体
            result = session.run("MATCH (e:__Entity__ {id: $id}) RETURN count(e) as count", id=target_id)
            target_exists = result.single()["count"] > 0
            
            print(f"实体检查: source={source_id} {'存在' if source_exists else '不存在'}, target={target_id} {'存在' if target_exists else '不存在'}")
            
            if not source_exists or not target_exists:
                print("部分实体不存在，将在导入过程中创建")
    
    # 5. 导入关系数据
    print("开始导入关系数据...")
    
    # 分两步执行，先创建关系节点和实体间关系，再创建与Chunk的关系
    # 步骤1: 创建关系节点和实体间关系
    relationship_statement = """
    // 创建关系节点
    MERGE (r:__Relationship__ {id: value.id})
    SET r.human_readable_id = value.human_readable_id,
        r.description = value.description,
        r.weight = value.weight,
        r.combined_degree = value.combined_degree,
        r.name = value.human_readable_id
    
    // 查找或创建源实体和目标实体
    WITH r, value
    MERGE (source:__Entity__ {id: value.source})
    MERGE (target:__Entity__ {id: value.target})
    
    // 创建RELATED关系
    MERGE (source)-[rel:RELATED]->(target)
    SET rel.relationship_id = value.id,
        rel.description = value.description,
        rel.weight = value.weight
    
    RETURN r.id as relationship_id
    """
    
    # 执行导入关系节点和实体间关系
    relationship_result = parallel_batched_import(relationship_statement, df_relationships, batch_size, max_workers)
    print(f"已创建 {relationship_result['successful_rows']} 个__Relationship__节点和RELATED关系")
    
    # 步骤2: 创建与Chunk的关系
    # 准备Chunk关系数据
    chunk_relations = []
    for _, row in df_relationships.iterrows():
        rel_id = row['id']
        for chunk_id in row['text_unit_ids']:
            chunk_relations.append({
                'relationship_id': rel_id,
                'chunk_id': chunk_id
            })
    
    if chunk_relations:
        df_chunk_relations = pd.DataFrame(chunk_relations)
        
        # 创建Chunk到Relationship的关系
        chunk_rel_statement = """
        MATCH (r:__Relationship__ {id: value.relationship_id})
        MATCH (c:__Chunk__ {id: value.chunk_id})
        MERGE (c)-[:HAS_RELATIONSHIP]->(r)
        """
        
        # 执行导入Chunk关系
        chunk_rel_result = parallel_batched_import(chunk_rel_statement, df_chunk_relations, batch_size, max_workers)
        print(f"已创建 {chunk_rel_result['successful_rows']} 个Chunk-Relationship关系")
    else:
        print("没有找到有效的Chunk-Relationship关系数据")
    
    # 6. 验证结果
    with driver.session(database=NEO4J_DATABASE) as session:
        # 检查Relationship节点数量
        result = session.run("MATCH (r:__Relationship__) RETURN count(r) as count")
        relationship_count = result.single()["count"]
        
        # 检查RELATED关系数量
        result = session.run("MATCH ()-[r:RELATED]->() RETURN count(r) as count")
        related_count = result.single()["count"]
        
        # 检查HAS_RELATIONSHIP关系数量
        try:
            result = session.run("MATCH (c:__Chunk__)-[r:HAS_RELATIONSHIP]->() RETURN count(r) as count")
            has_relationship_count = result.single()["count"]
        except Exception as e:
            print(f"查询HAS_RELATIONSHIP关系时出错: {e}")
            has_relationship_count = "未知"
        
        print(f"验证结果: {relationship_count} 个__Relationship__节点, {related_count} 个RELATED关系, {has_relationship_count} 个HAS_RELATIONSHIP关系")
    
    return relationship_result

In [166]:
# 导入关系
import_relationships(df_relations, batch_size=100, max_workers=8)

已创建__Relationship__.id唯一性约束
节点数量: 45 个__Entity__节点, 6 个__Chunk__节点
预处理text_unit_ids...
验证source和target实体...
实体检查: source=APPLE INC. 不存在, target=STEVE JOBS 不存在
部分实体不存在，将在导入过程中创建
实体检查: source=APPLE INC. 不存在, target=STEVE WOZNIAK 不存在
部分实体不存在，将在导入过程中创建
实体检查: source=APPLE INC. 不存在, target=RON WAYNE 不存在
部分实体不存在，将在导入过程中创建
实体检查: source=APPLE INC. 不存在, target=CUPERTINO 不存在
部分实体不存在，将在导入过程中创建
实体检查: source=APPLE INC. 不存在, target=SILICON VALLEY 不存在
部分实体不存在，将在导入过程中创建
开始导入关系数据...
开始并行导入 72 行数据，分为 1 个批次，每批 100 条
批次 0 完成: 72 行, 耗时 0.30秒
进度: 1/1 批次完成 (100.0%)
导入完成: 总计 72 行, 成功 72 行, 失败 0 行
总耗时: 0.30秒, 平均速度: 239.11 行/秒
已创建 72 个__Relationship__节点和RELATED关系
开始并行导入 104 行数据，分为 2 个批次，每批 100 条
批次 0 完成: 100 行, 耗时 0.13秒
进度: 1/2 批次完成 (50.0%)
批次 1 完成: 4 行, 耗时 2.10秒
进度: 2/2 批次完成 (100.0%)
导入完成: 总计 104 行, 成功 104 行, 失败 0 行
总耗时: 2.10秒, 平均速度: 49.42 行/秒
已创建 104 个Chunk-Relationship关系
验证结果: 72 个__Relationship__节点, 72 个RELATED关系, 104 个HAS_RELATIONSHIP关系


{'total_rows': 72,
 'successful_rows': 72,
 'failed_rows': 0,
 'duration_seconds': 0.3011190891265869,
 'rows_per_second': 239.1080559151534,
 'batch_results': [{'batch': 0,
   'rows': 72,
   'success': True,
   'duration': 0.3001217842102051,
   'counters': {'_contains_updates': True, 'labels_added': 117, 'relationships_created': 72, 'nodes_created': 117, 'properties_set': 693}}]}

&emsp;&emsp;执行完成后，可以在`localhost:7474`的`Neo4j`浏览器中，可以看到会生成`__Relationship__`节点，同时，`__Entity__`节点与`__Chunk__`节点之间，也创建了`HAS_RELATIONSHIP`关系，以及`__Relationship__`节点与`__Entity__`节点之间，也创建了`RELATED`关系。如下图所示：


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503121928544.png" width=80%></div>

&emsp;&emsp;至此，我们就完成了`relationships.parquet`文件的导入。接下来，我们开始导入`communities.parquet`文件。

## 4.6 导入社区

&emsp;&emsp;在`communities.parquet`文件中，主要存储的通过莱顿社区发现算法，根据`settings.yaml` 文件中`cluster_graph`配置划分出的社区层级，同时通过后处理，对每个社区中包含的实体，关系，文本单元等进行关联。读取该文件的代码如下所示：

In [167]:
# 使用绝对路径读取 Parquet 文件
df_communities = pd.read_parquet('D:\\agent\graphrag\graphrag\output\communities.parquet')  # 替换为实际路径

# 假设 df 是你的 DataFrame
print(tabulate(df_communities, headers='keys', tablefmt='pretty', showindex=False, stralign='left', maxcolwidths=[20, 20, 20, 20, 20]))

+----------------------+-------------------+-----------+-------+--------+----------+-------------+------------------------------------------+------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------+------------+------+
| id                   | human_readable_id | community | level | parent | children | title       | entity_ids                               | relationship_ids                         | text_unit_ids                                                                                                                        | period     | size |
+----------------------+-------------------+-----------+-------+--------+----------+-------------+------------------------------------------+------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------+-

<>:2: SyntaxWarning: invalid escape sequence '\g'
<>:2: SyntaxWarning: invalid escape sequence '\g'
C:\Users\Administrator\AppData\Local\Temp\ipykernel_23992\2920323793.py:2: SyntaxWarning: invalid escape sequence '\g'
  df_communities = pd.read_parquet('D:\\agent\graphrag\graphrag\output\communities.parquet')  # 替换为实际路径


&emsp;&emsp;各个字段的解释如下所示：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>communities DataFrame 字段解释</font></p>
<div class="center">

| 字段名              | 类型               | 说明                                                         |
|---------------------|--------------------|--------------------------------------------------------------|
| id                  | 字符串             | 唯一标识符，用于唯一标识每个社区记录。                       |
| human_readable_id   | 字符串             | 可读的标识符，通常用于用户界面显示，便于用户理解和识别。   |
| community           | 整数               | 社区的标识符，表示该记录所属的社区编号。                   |
| level               | 整数               | 社区的层级，表示该社区在层级结构中的位置。                 |
| parent              | 整数               | 父社区的标识符，表示该社区的上级社区，-1 表示没有父社区。  |
| children            | 列表（字符串）     | 子社区的标识符列表，表示该社区下属的子社区。               |
| title               | 字符串             | 社区的名称或标题                  |
| entity_ids          | 列表（字符串）     | 关联的实体的唯一标识符列表，通常用于将社区与实体关联。     |
| relationship_ids    | 列表（字符串）     | 关联的关系的唯一标识符列表，表示该社区中涉及的关系。       |
| text_unit_ids       | 列表（字符串）     | 关联的文本单元的唯一标识符列表，通常用于将社区与文本关联。 |
| period              | 日期               | 社区的时间段，表示该社区的有效时间范围。                   |
| size                | 整数               | 社区的规模，通常表示社区中包含的实体或成员数量。           |
</div>

&emsp;&emsp;思考现在已经完成的节点和关系，我们其实只需要把关系与社区进行关联就足够了。因此，导入的策略可以考虑以下原则：

- **节点标识**：需要使用`id`字段作为Community节点的唯一标识符，确保图谱中节点一致性。
- **约束策略**：唯一性约束，确保关系的id是唯一的。
- **属性选择**：比如社区的名称，级别，规模等，根据需求选择。
- **关系策略**：与该社区相关的实体节点（通过 relationship_ids）与社区建立 IN_COMMUNITY 关系。

&emsp;&emsp;因此，把上述的策略转化成代码。就如下所示：

In [168]:
def setup_community_constraints():
    """创建Community标签的约束"""
    with driver.session(database=NEO4J_DATABASE) as session:
        try:
            # 创建Community.id唯一性约束
            session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (c:__Community__) REQUIRE c.id IS UNIQUE")
            print("已创建__Community__.id唯一性约束")
        except Exception as e:
            print(f"创建__Community__约束时出错 (可能已存在): {e}")
            # 尝试旧版本Neo4j的语法
            try:
                session.run("CREATE CONSTRAINT ON (c:__Community__) ASSERT c.id IS UNIQUE")
                print("已使用旧语法创建__Community__.id唯一性约束")
            except Exception as e2:
                print(f"使用旧语法创建约束也失败: {e2}")

def import_communities(df_communities, batch_size=100, max_workers=8):
    """
    导入社区(Community)数据到Neo4j
    
    参数:
    - df_communities: 包含社区数据的DataFrame
    - batch_size: 每批处理的行数
    - max_workers: 并行线程数
    
    返回:
    - 导入统计信息的字典
    """
    # 1. 创建Community的约束
    setup_community_constraints()
    
    # 2. 预处理列表字段 - 确保是列表格式
    print("预处理列表字段...")
    
    # 创建DataFrame的副本
    df_communities = df_communities.copy()
    
    # 需要处理的列表字段
    list_fields = ['children', 'entity_ids', 'relationship_ids', 'text_unit_ids']
    
    for field in list_fields:
        if field in df_communities.columns:
            for idx, row in df_communities.iterrows():
                field_value = row.get(field)
                
                # 如果不是列表，转换为列表
                if not isinstance(field_value, list):
                    if isinstance(field_value, str):
                        try:
                            # 尝试解析JSON字符串
                            import json
                            field_value = json.loads(field_value)
                        except:
                            # 如果解析失败，将其作为单个元素的列表
                            field_value = [field_value]
                    elif hasattr(field_value, 'dtype') and hasattr(field_value, 'tolist'):
                        # 处理NumPy数组
                        field_value = field_value.tolist()
                    else:
                        # 其他类型，转为列表
                        field_value = [field_value] if field_value is not None else []
                
                # 处理嵌套列表
                flat_field_value = []
                for item in field_value:
                    if isinstance(item, list) or (hasattr(item, 'dtype') and hasattr(item, 'tolist')):
                        if hasattr(item, 'tolist'):
                            flat_field_value.extend(item.tolist())
                        else:
                            flat_field_value.extend(item)
                    else:
                        flat_field_value.append(item)
                
                # 确保所有ID都是字符串且非空
                if field in ['entity_ids', 'relationship_ids', 'text_unit_ids']:
                    flat_field_value = [str(id) for id in flat_field_value if id is not None and str(id).strip() != '']
                
                # 更新DataFrame
                df_communities.at[idx, field] = flat_field_value
    
    # 3. 导入社区节点
    print("开始导入社区节点...")
    
    community_statement = """
    // 创建Community节点
    MERGE (c:__Community__ {id: value.id})
    SET c.human_readable_id = value.human_readable_id,
        c.community = value.community,
        c.level = value.level,
        c.parent = value.parent,
        c.children = value.children,
        c.title = value.title,
        c.period = value.period,
        c.size = value.size,
        c.name = coalesce(value.title, value.human_readable_id, 'Community_' + value.id)
    
    RETURN c.id as community_id
    """
    
    # 执行导入社区节点
    community_result = parallel_batched_import(community_statement, df_communities, batch_size, max_workers)
    print(f"已创建 {community_result['successful_rows']} 个__Community__节点")
    
    # 4. 创建与Entity的关系
    print("开始创建社区与实体的关系...")
    
    # 准备Entity关系数据
    entity_relations = []
    for _, row in df_communities.iterrows():
        community_id = row['id']
        entity_ids = row.get('entity_ids', [])
        
        for entity_id in entity_ids:
            entity_relations.append({
                'community_id': community_id,
                'entity_id': entity_id
            })
    
    if entity_relations:
        df_entity_relations = pd.DataFrame(entity_relations)
        
        entity_rel_statement = """
        MATCH (c:__Community__ {id: value.community_id})
        MATCH (e:__Entity__ {id: value.entity_id})
        MERGE (e)-[:IN_COMMUNITY]->(c)
        """
        
        entity_rel_result = parallel_batched_import(entity_rel_statement, df_entity_relations, batch_size, max_workers)
        print(f"已创建 {entity_rel_result['successful_rows']} 个Entity-Community关系")
    else:
        print("没有找到有效的Entity-Community关系数据")
    
    # 5. 创建与Relationship的关系
    print("开始创建社区与关系的关系...")
    
    # 准备Relationship关系数据
    rel_relations = []
    for _, row in df_communities.iterrows():
        community_id = row['id']
        relationship_ids = row.get('relationship_ids', [])
        
        for rel_id in relationship_ids:
            rel_relations.append({
                'community_id': community_id,
                'relationship_id': rel_id
            })
    
    if rel_relations:
        df_rel_relations = pd.DataFrame(rel_relations)
        
        rel_rel_statement = """
        MATCH (c:__Community__ {id: value.community_id})
        MATCH (r:__Relationship__ {id: value.relationship_id})
        MERGE (r)-[:IN_COMMUNITY]->(c)
        """
        
        rel_rel_result = parallel_batched_import(rel_rel_statement, df_rel_relations, batch_size, max_workers)
        print(f"已创建 {rel_rel_result['successful_rows']} 个Relationship-Community关系")
    else:
        print("没有找到有效的Relationship-Community关系数据")
    
    # 6. 验证结果
    with driver.session(database=NEO4J_DATABASE) as session:
        # 检查Community节点数量
        result = session.run("MATCH (c:__Community__) RETURN count(c) as count")
        community_count = result.single()["count"]
        
        # 检查IN_COMMUNITY关系数量
        result = session.run("MATCH ()-[r:IN_COMMUNITY]->() RETURN count(r) as count")
        in_community_count = result.single()["count"]
        
        print(f"验证结果: {community_count} 个__Community__节点, {in_community_count} 个IN_COMMUNITY关系")
    
    return community_result

In [169]:
# 导入社区
import_communities(df_communities, batch_size=100, max_workers=8)

已创建__Community__.id唯一性约束
预处理列表字段...
开始导入社区节点...
开始并行导入 8 行数据，分为 1 个批次，每批 100 条
批次 0 完成: 8 行, 耗时 0.11秒
进度: 1/1 批次完成 (100.0%)
导入完成: 总计 8 行, 成功 8 行, 失败 0 行
总耗时: 0.11秒, 平均速度: 76.19 行/秒
已创建 8 个__Community__节点
开始创建社区与实体的关系...
开始并行导入 59 行数据，分为 1 个批次，每批 100 条
批次 0 完成: 59 行, 耗时 0.08秒
进度: 1/1 批次完成 (100.0%)
导入完成: 总计 59 行, 成功 59 行, 失败 0 行
总耗时: 0.08秒, 平均速度: 728.45 行/秒
已创建 59 个Entity-Community关系
开始创建社区与关系的关系...
开始并行导入 62 行数据，分为 1 个批次，每批 100 条
批次 0 完成: 62 行, 耗时 0.10秒
进度: 1/1 批次完成 (100.0%)
导入完成: 总计 62 行, 成功 62 行, 失败 0 行
总耗时: 0.10秒, 平均速度: 624.35 行/秒
已创建 62 个Relationship-Community关系
验证结果: 8 个__Community__节点, 121 个IN_COMMUNITY关系


{'total_rows': 8,
 'successful_rows': 8,
 'failed_rows': 0,
 'duration_seconds': 0.10500097274780273,
 'rows_per_second': 76.18977034826955,
 'batch_results': [{'batch': 0,
   'rows': 8,
   'success': True,
   'duration': 0.10500097274780273,
   'counters': {'_contains_updates': True, 'labels_added': 8, 'nodes_created': 8, 'properties_set': 80}}]}

&emsp;&emsp;执行完成后，可以在`localhost:7474`的`Neo4j`浏览器中，可以看到会生成`__Community__`节点，同时，`__Entity__`节点与`__Community__`节点之间，也创建了`IN_COMMUNITY`关系，以及`__Relationship__`节点与`__Community__`节点之间，也创建了`IN_COMMUNITY`关系。如下图所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503121942902.png" width=80%></div>

&emsp;&emsp;至此，我们就完成了`communities.parquet`文件的导入。接下来，我们开始导入`community_reports.parquet`文件。

## 4.7 导入社区报告

&emsp;&emsp;在`community_reports.parquet`文件中，主要存储的内容是让大模型通过提示工程，对每个社区中的实体和关系进行总结，并生成社区的报告。读取该文件的代码如下所示：

In [170]:
# 使用绝对路径读取 Parquet 文件
df_communities_reports = pd.read_parquet('D:\\agent\graphrag\graphrag\output\community_reports.parquet')  # 替换为实际路径

# 假设 df 是你的 DataFrame
print(tabulate(df_communities_reports, headers='keys', tablefmt='pretty', showindex=False, stralign='left', maxcolwidths=[20, 20, 20, 20, 20]))

<>:2: SyntaxWarning: invalid escape sequence '\g'
<>:2: SyntaxWarning: invalid escape sequence '\g'
C:\Users\Administrator\AppData\Local\Temp\ipykernel_23992\1065169428.py:2: SyntaxWarning: invalid escape sequence '\g'
  df_communities_reports = pd.read_parquet('D:\\agent\graphrag\graphrag\output\community_reports.parquet')  # 替换为实际路径


+----------------------+-------------------+-----------+-------+--------+----------+-----------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

&emsp;&emsp;各个字段的解释如下所示：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>community_reports DataFrame 字段解释</font></p>
<div class="center">

| 字段名              | 类型               | 说明                                                         |
|---------------------|--------------------|--------------------------------------------------------------|
| id                  | 字符串             | 唯一标识符，用于唯一标识每个记录。                           |
| human_readable_id   | 字符串             | 可读的标识符，通常用于用户界面显示，便于用户理解和识别。   |
| community           | 整数               | 社区的标识符，表示该记录所属的社区编号。                   |
| level               | 整数               | 社区的层级，表示该社区在层级结构中的位置。                 |
| parent              | 整数               | 父社区的标识符，表示该社区的上级社区，-1 表示没有父社区。  |
| children            | 列表（字符串）     | 子社区的标识符列表，表示该社区下属的子社区。               |
| title               | 字符串             | 社区的名称或标题，简要描述该社区的主题。                   |
| summary             | 字符串             | 对社区的简要描述，提供更多上下文信息。                     |
| full_content        | 字符串             | 社区的详细内容，包含更全面的信息和背景。                   |
| rank                | 浮点数             | 社区的排名，通常用于表示社区的重要性或影响力。             |
| rating_explanation   | 字符串             | 对社区评分的解释，说明评分的依据和原因。                   |
| findings            | 列表（字典）       | 相关发现的列表，每个发现包含解释和摘要。                   |
| full_content_json   | JSON               | 社区的完整内容，以 JSON 格式存储，便于数据交换和处理。     |
| period              | 日期               | 社区的时间段，表示该社区的有效时间范围。                   |
| size                | 整数               | 社区的规模，通常表示社区中包含的实体或成员数量。           |
</div>

&emsp;&emsp;对于社区报告，是不是需要将它们连接到它们所涉及的实体，并且为社区更新属性。因此，导入的策略可以考虑以下原则：

- **属性策略**：更新社区的属性，包括 level、title、rank、rank_explanation、full_content 和 summary。
- **关系策略**：遍历与该社区相关的发现（findings），为每个发现创建一个新的 Finding 节点，并将其与社区建立 HAS_FINDING 关系。

In [171]:
def import_community_reports(df_reports, batch_size=20, max_workers=2):
    """
    导入社区报告数据到Neo4j
    
    参数:
    - df_reports: 包含社区报告数据的DataFrame
    - batch_size: 每批处理的行数
    - max_workers: 并行线程数
    
    返回:
    - 导入统计信息的字典
    """
    # 1. 预处理社区报告数据
    print("预处理社区报告数据...")
    df_reports = df_reports.copy()
    
    # 先创建新列，避免KeyError
    df_reports['community_str'] = None
    df_reports['processed_findings'] = None
    
    # 检查findings字段并打印样本数据以便调试
    sample_findings = []
    
    for idx, row in df_reports.iterrows():
        # 处理community字段 - 先转换为字符串再赋值
        if 'community' in row:
            community_str = str(row['community'])
            df_reports.at[idx, 'community_str'] = community_str
        
        # 处理findings字段 - 特别处理NumPy数组
        findings = row.get('findings')
        
        # 处理NumPy数组
        if hasattr(findings, 'dtype') and hasattr(findings, 'tolist'):
            print(f"行 {idx}: 将NumPy数组转换为列表，形状: {findings.shape if hasattr(findings, 'shape') else 'unknown'}")
            try:
                findings = findings.tolist()
            except Exception as e:
                print(f"行 {idx}: 转换NumPy数组失败: {e}")
                findings = []
        # 处理其他类型
        elif not isinstance(findings, list):
            if isinstance(findings, str):
                try:
                    import json
                    findings = json.loads(findings)
                    print(f"行 {idx}: 从字符串解析findings成功")
                except Exception as e:
                    print(f"行 {idx}: 解析findings字符串失败: {e}")
                    findings = []
            else:
                print(f"行 {idx}: findings不是列表、NumPy数组或字符串，而是 {type(findings)}")
                findings = []
        
        # 确保findings是有效的列表
        if not isinstance(findings, list):
            print(f"行 {idx}: 处理后findings仍不是列表，强制设为空列表")
            findings = []
        
        # 打印findings的内容以便调试
        if len(findings) > 0:
            print(f"行 {idx}: 找到 {len(findings)} 个findings")
            if idx < 3:  # 只打印前3行的详细内容
                print(f"  示例: {findings[0]}")
                sample_findings.append((idx, findings))
        else:
            print(f"行 {idx}: 没有找到findings")
        
        # 确保每个finding是字典
        valid_findings = []
        for i, finding in enumerate(findings):
            if isinstance(finding, dict):
                # 确保必要的字段存在
                if 'summary' not in finding:
                    finding['summary'] = f"Finding_{i}"
                if 'explanation' not in finding:
                    finding['explanation'] = ""
                valid_findings.append(finding)
            else:
                print(f"行 {idx}: 跳过无效的finding类型: {type(finding)}")
                # 尝试转换为字典
                if isinstance(finding, (list, tuple)) and len(finding) >= 2:
                    try:
                        valid_findings.append({
                            'summary': str(finding[0]),
                            'explanation': str(finding[1])
                        })
                        print(f"  成功转换为字典: {finding[0]}")
                    except Exception as e:
                        print(f"  转换失败: {e}")
        
        # 更新DataFrame - 使用at而不是loc
        df_reports.at[idx, 'processed_findings'] = valid_findings
    
    # 打印样本数据
    print(f"收集了 {len(sample_findings)} 个样本findings:")
    for idx, findings in sample_findings:
        print(f"行 {idx}: {len(findings)} 个findings")
    
    # 2. 准备Finding数据 - 直接从处理后的数据创建
    print("准备Finding数据...")
    findings_data = []
    
    for idx, row in df_reports.iterrows():
        community_str = row['community_str']
        processed_findings = row['processed_findings']
        
        if not isinstance(processed_findings, list):
            print(f"行 {idx}: processed_findings不是列表，跳过")
            continue
            
        for i, finding in enumerate(processed_findings):
            if isinstance(finding, dict):
                finding_id = f"{community_str}_{i}"
                findings_data.append({
                    'finding_id': finding_id,
                    'community_id': community_str,
                    'summary': finding.get('summary', f"Finding_{i}"),
                    'explanation': finding.get('explanation', "")
                })
    
    print(f"准备了 {len(findings_data)} 个Finding数据")
    
    # 3. 导入社区节点
    print("步骤1: 导入社区节点...")
    
    # 社区节点创建语句
    community_statement = """
    MERGE (c:__Community__ {community: value.community_str})
    SET c.level = value.level,
        c.title = value.title,
        c.rank = value.rank,
        c.rating_explanation = value.rating_explanation,
        c.full_content = value.full_content,
        c.summary = value.summary,
        c.name = coalesce(value.title, 'Community_' + value.community_str)
    RETURN c.community as community_id
    """
    
    # 执行导入社区节点
    community_result = parallel_batched_import(community_statement, df_reports, batch_size, max_workers)
    print(f"已创建/更新 {community_result['successful_rows']} 个社区节点")
    
    # 4. 如果有Finding数据，创建Finding节点和关系
    if findings_data:
        print("步骤2: 导入Finding节点和关系...")
        df_findings = pd.DataFrame(findings_data)
        
        finding_statement = """
        // 创建Finding节点
        MERGE (f:__Finding__ {id: value.finding_id})
        SET f.summary = value.summary,
            f.explanation = value.explanation,
            f.name = value.summary
        
        // 创建与Community的关系
        WITH f, value
        MATCH (c:__Community__ {community: value.community_id})
        MERGE (c)-[:HAS_FINDING]->(f)
        """
        
        # 执行导入Finding节点和关系
        finding_result = parallel_batched_import(finding_statement, df_findings, batch_size, max_workers)
        print(f"已创建 {finding_result['successful_rows']} 个Finding节点和HAS_FINDING关系")
    else:
        print("没有找到有效的Finding数据，跳过创建Finding节点和关系")
    
    # 5. 验证结果
    with driver.session(database=NEO4J_DATABASE) as session:
        # 检查Community节点数量
        result = session.run("MATCH (c:__Community__) RETURN count(c) as count")
        community_count = result.single()["count"]
        
        # 检查Finding节点数量 - 使用try-except处理可能的错误
        try:
            result = session.run("MATCH (f:__Finding__) RETURN count(f) as count")
            finding_count = result.single()["count"]
        except Exception as e:
            print(f"查询Finding节点时出错: {e}")
            finding_count = 0
        
        # 检查HAS_FINDING关系数量 - 使用try-except处理可能的错误
        try:
            result = session.run("MATCH ()-[r:HAS_FINDING]->() RETURN count(r) as count")
            has_finding_count = result.single()["count"]
        except Exception as e:
            print(f"查询HAS_FINDING关系时出错: {e}")
            has_finding_count = 0
        
        print(f"验证结果: {community_count} 个__Community__节点, {finding_count} 个__Finding__节点, {has_finding_count} 个HAS_FINDING关系")
    
    return community_result

In [172]:
# 导入社区报告
import_community_reports(df_communities_reports, batch_size=100, max_workers=8)


预处理社区报告数据...
行 0: 将NumPy数组转换为列表，形状: (5,)
行 0: 找到 5 个findings
  示例: {'explanation': "Apple Inc. stands as a central entity in this community with substantial influence in the global technology sector. The company has grown to become one of the world's most valuable entities, with a market capitalization exceeding $2 trillion as of 2023. This financial magnitude underscores Apple's significant economic impact and market dominance. The company's high degree of connectivity (degree 17) indicates extensive relationships and influence within the technology ecosystem. [Data: Entities (36)]", 'summary': "Apple's dominant position in the technology industry"}
行 1: 将NumPy数组转换为列表，形状: (6,)
行 1: 找到 6 个findings
  示例: {'explanation': "Steve Jobs serves as the central entity in this community with the highest degree of connectivity. As co-founder of Apple who returned in 1997, he played a crucial role in the company's resurgence and product innovation. His leadership directly influenced the developmen

{'total_rows': 8,
 'successful_rows': 8,
 'failed_rows': 0,
 'duration_seconds': 0.09967803955078125,
 'rows_per_second': 80.2584003061615,
 'batch_results': [{'batch': 0,
   'rows': 8,
   'success': True,
   'duration': 0.0976862907409668,
   'counters': {'_contains_updates': True, 'labels_added': 8, 'nodes_created': 8, 'properties_set': 64}}]}

&emsp;&emsp;在完成后，可以在`localhost:7474`的`Neo4j`浏览器中，可以看到会生成`__Community__`节点，同时，`__Finding__`节点与`__Community__`节点之间，也创建了`HAS_FINDING`关系。如下图所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503121957586.png" width=80%></div>

&emsp;&emsp;至此，我们就完成了`community_reports.parquet`文件的导入。


&emsp;&emsp;完成了所有文件的导入后，一个完整的知识图谱就构建完成了。现阶段就已经可以开始使用`Neo4j`的`Cypher`语句，查询知识图谱中的数据。比如一些常见的查询，如下所示：


&emsp;&emsp;显示一些__Entity__节点及其关系:

```bash
    MATCH path = (:__Entity__)-[:RELATED]->(:__Entity__)
    RETURN path LIMIT 200
```


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503131154423.png" width=80%></div>

&emsp;&emsp;显示块和文档（词汇图）：

```bash
    MATCH (d:__Document__) WITH d LIMIT 1
    MATCH path = (d)<-[:PART_OF]-(c:__Chunk__)
    RETURN path LIMIT 100
```

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503131154424.png" width=80%></div>

&emsp;&emsp;返回最多 25 个实体之间的社区关系路径：

```bash
    MATCH p=()-[r:IN_COMMUNITY]->() 
    RETURN p LIMIT 25
```

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503131154425.png" width=80%></div>

&emsp;&emsp;至此，我们完成了`Microsoft GraphRAG` 生成索引的完整知识图谱构建与可视化。详细了解了知识图谱的全貌后，接下来的课程，我们进行`Microsoft GraphRAG` 实现的`Query` 阶段的实操以及底层原理的讲解。
